# Jupyter Notebook for the evaluation of nucleon PDFs from Distillation correlation functions

## Initialization

In [ ]:
import numpy as np
import scipy.linalg as la
import scipy.optimize as scipyOpt
import math, itertools as it
import matplotlib as mpl ; import matplotlib.pyplot as plt ; plt.ion()
from matplotlib import rc
import matplotlib.gridspec as gridspec
import pickle
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetiva']})
rc('text', usetex=True)

print ("Done")

## Directories, Parameters, Constants

### Generic definitions

In [ ]:
def dispTag(d):
    dL = lambda i: ("z" if i != 0 else "") + ("+" if i > 0 else "") + str(i)
    return 'disp_%s'%(dL(d))

def MomTag(zMom):
    mSgn = lambda i: ("+" if i > 0 else "") + str(i)
    return 'pz.%s'%(mSgn(zMom))

def phMomTpl(phTag,zMom):
    return (phTag,MomTag(zMom))

def momFileTag(mVec):
    mFTag = 'momXYZ.%d.%d.%d' % (mVec[0],mVec[1],mVec[2])
    return mFTag
#-----------------------------------

phaseTypes    = ['unphased']
SrcSnkOpTypes = ['single']
maxDisp = 8
dispLen = np.arange(-maxDisp,maxDisp+1)

dispLen_ave = np.arange(maxDisp+1)

# Parameters for unphased correlators with a single src-sink operator
phTS = 'unphased'
opTS = 'single'

# Note: Every combination of momentum, t0 and t_sink is considered a 'dataset'
ZmomVal   = {(phTS,opTS): [0,1,2,3,-1,-2,-3]} # Available momenta values
TsinkList = {(phTS,opTS): [4,6,8,10,12,14]}   # Tsink Values
T0List    = {(phTS,opTS): [0,16,32,48]}       # T0 Values

ZmomVal_ave = {(phTS,opTS): [0,1,2,3]}

# Insertion gamma matrices
gMatList = ['gt',
            'gxg5','gyg5','gzg5','gtg5',
            'gxgy','gxgz','gxgt','gygz','gygt','gzgt']

# (Maximum) Number of configs
Ncfg_all  = 349

# Base Directory of two- and three-point correlation functions
corr2ptWorkDir = '../2pt_corr_effEnergy'
corr3ptMainDir = {(phTS,opTS): '3pt_data_%s'%(phTS)}
corr2ptMainDir = {(phTS,opTS): '%s/2pt_data_%s'%(corr2ptWorkDir,phTS)}

momTags_Dict = {(phTS,opTS): []}
momVector_Dict  = {}
Ncfg_Dict_3pt = {}
Ncfg_Dict_2pt = {}
existsDataSet_3pt = {}
existsDataSet_2pt = {}
Nt_Dict_2pt = {}
for zmom in ZmomVal[(phTS,opTS)]:
    mTag = MomTag(zmom)
    momTags_Dict[(phTS,opTS)].append(mTag)
    momVector_Dict[mTag] = np.array([0,0,zmom])
    
    for tsnk in TsinkList[(phTS,opTS)]:
        dTpl = (phTS,opTS,mTag,tsnk)
        Ncfg_Dict_3pt[dTpl] = Ncfg_all  # All datasets have the same number of configs
        for t0 in T0List[(phTS,opTS)]:
            dTpl3pt = (phTS,opTS,mTag,tsnk,t0)
            existsDataSet_3pt[dTpl3pt] = True  # Whether all possible datasets exist for the three-point function

    dTpl2 = (phTS,opTS,mTag)
    Ncfg_Dict_2pt[dTpl2] = Ncfg_all            
    for t0 in T0List[(phTS,opTS)]:
        dTpl2pt = (phTS,opTS,mTag,t0)
        existsDataSet_2pt[dTpl2pt] = True  # Whether all possible datasets exist for the two-point function
        
    Nt_Dict_2pt[phMomTpl(phTS,zmom)] = 21 # 2pt for all momenta have 21 time-slices        
#----------------------------------------------

momTags_Dict_ave = {(phTS,opTS): []}
for zmom in ZmomVal_ave[(phTS,opTS)]:
    mTag = MomTag(zmom)
    momTags_Dict_ave[(phTS,opTS)].append(mTag)
    

# Zero-momentum for t0=48 does not exist, skip it
zmom = 0
t0 = 48
mTag = MomTag(zmom)
dTpl2pt = (phTS,opTS,mTag,t0)
existsDataSet_2pt[dTpl2pt] = False
#----------------------------------------------


# Which datasets to analyze (if there's at least one t0 for each momentum and tsink that's OK)
analyzeDataSetM = {}
for mTag in momTags_Dict[(phTS,opTS)]:
    for tsnk in TsinkList[(phTS,opTS)]:
        adTplM = (phTS,opTS,mTag,tsnk)
        for t0 in T0List[(phTS,opTS)]:
            dTpl3pt = (phTS,opTS,mTag,tsnk,t0)
            dTpl2pt = (phTS,opTS,mTag,t0)
            if existsDataSet_3pt[dTpl3pt] and existsDataSet_2pt[dTpl2pt]:
                analyzeDataSetM[adTplM] = True
            else:
                analyzeDataSetM[adTplM] = False
            break

analyzeDataSetE = {}
for mTag in momTags_Dict[(phTS,opTS)]:
    adTplE = (phTS,opTS,mTag)
    for t0 in T0List[(phTS,opTS)]:
        dTpl2pt = (phTS,opTS,mTag,t0)
        if existsDataSet_2pt[dTpl2pt]:
            analyzeDataSetE[adTplE] = True
        else:
            analyzeDataSetE[adTplE] = False
        break        
#----------------------------------------------

ReIm = ['Re','Im']

print('Momentum Tags:\n', momTags_Dict)
print('\n')
print('Momentum Vectors:\n', momVector_Dict)

print('\nDone\n')

### Read/define source-insertion-sink operators

In [ ]:

# Interpolating operators  for 3-point functions         
nRow3pt_Dict = {'single': {}}
nRow2pt_Dict = {'single': {}}
srcOpList3pt_Dict = {'single': {}}
snkOpList3pt_Dict = {'single': {}}
srcOpList2pt_Dict = {'single': {}}
snkOpList2pt_Dict = {'single': {}}
for zmom in ZmomVal[(phTS,opTS)]:
    pmTpl = phMomTpl(phTS,zmom)

    # 2-point function interpolators have one row for zmom = 0
    if zmom == 0:
        nRow2pt_Dict['single'][pmTpl] = 1
    else:
        nRow2pt_Dict['single'][pmTpl] = 2
    
    # 3-point function interpolators have two-rows for all momenta
    nRow3pt_Dict['single'][pmTpl] = 2

    if zmom == 0:
        srcOpList3pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_G1g1']
        snkOpList3pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_G1g1']
        srcOpList2pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_G1g1']
        snkOpList2pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_G1g1']
    else:
        srcOpList3pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1']
        snkOpList3pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1']
        srcOpList2pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1']
        snkOpList2pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1']
#----------------------------------------------


# Insertion operators list, that's a common file among all momenta
insOpFile_Dict = {phTS: '../pPDF/insert_gamma_row.DA.list'}

# Read insertion operators
insOptrList = {}
insOpFname = insOpFile_Dict[phTS]
insOptrList[phTS] = {'name': [], 'row': []}
with open(insOpFname) as fp:
    line = fp.readlines()
    for t in line:
        insOptrList[phTS]['name'].append(t.split()[0])
        insOptrList[phTS]['row'].append(int(t.split()[1]))

# Create dictionary to map actual gamma matrices to operator names and rows
gMatMap = {phTS: {'gt'  : ('b_b0xDA__J0_A1pP', 1),
                  'gxg5': ('a_a1xDA__J1_T1pM', 1),
                  'gyg5': ('a_a1xDA__J1_T1pM', 3),
                  'gzg5': ('a_a1xDA__J1_T1pM', 2),
                  'gtg5': ('pion_pion_2xDA__J0_A1mM', 1),
                  'gxgy': ('b_b1xDA__J1_T1pP', 2),
                  'gxgz': ('b_b1xDA__J1_T1pP', 3),
                  'gxgt': ('rho_rho_2xDA__J1_T1mP', 3),
                  'gygz': ('b_b1xDA__J1_T1pP', 1),
                  'gygt': ('rho_rho_2xDA__J1_T1mP', 1),
                  'gzgt': ('rho_rho_2xDA__J1_T1mP', 2)}}
#----------------------------------------------


# Create dictionary to map actual gamma matrices to latex format
gMatLatex = {'gt'  : '\gamma_t',
             'gxg5': '\gamma_x\gamma_5',
             'gyg5': '\gamma_y\gamma_5',
             'gzg5': '\gamma_z\gamma_5',
             'gtg5': '\gamma_t\gamma_5',
             'gxgy': '\gamma_x\gamma_y',
             'gxgz': '\gamma_x\gamma_z',
             'gxgt': '\gamma_x\gamma_t',
             'gygz': '\gamma_y\gamma_z',
             'gygt': '\gamma_y\gamma_t',
             'gzgt': '\gamma_z\gamma_t'}
#----------------------------------------------



print('Insertion operator List:\n', insOptrList[phTS])

print('\nDone\n')

### Ensemble-related definitions

In [ ]:
# Ensemble parameters
L = 32
T = 64

binsize=1
#------------------------------------------------


Pi = np.pi

uM = 2*Pi/L  # Unit momentum

# Nucleon, pion mass
alat_fm = 0.094
alat_iGeV = alat_fm / 0.1973
mpi_GeV = 0.350
mN_GeV = 1.123
amN = mN_GeV * alat_iGeV

# Dispersion relation
def dispRel(am,k):
    ap = uM * k
    return np.sqrt(am**2 + np.dot(ap,ap))

print('Done\n')

### Plotting dictionaries

In [ ]:
pDct   = {0: 'ro', 1: 'bs', 2: 'g^', 3: 'mp', 4: 'cX'}
cDct   = {0: 'r', 1: 'b', 2: 'g', 3: 'm', 4: 'c', 5: 'darkorange', 6: 'y', 7: 'gold'}
mDct   = {0: 'o', 1: 's', 2: '^', 3: 'p', 4: 'X', 5: 'd', 6: 'v', 7: '>'}
osDct  = {0: 0.0 , 1: 0.15, 2: 0.3, 3: 0.45, 4: 0.6}
osDctF = {0: 0.0 , 1: 0.07, 2: 0.15, 3: 0.22, 4: 0.3,
          5: 0.37, 6: 0.45, 7: 0.52, 8: 0.60, 9: 0.67}
fsDct = {1: 'full',2:'none'}


### Jackknife-related functions

In [ ]:
def jackknife_ave(bins,Nbins,tL):

    Jax = 0
    
    if(tL==1):
        ave = np.mean(bins,dtype=bins.dtype)

        sqsum = sum(map(lambda x:x*x,ave-bins))
        fac = (Nbins -1) / float(Nbins)
        err = np.sqrt(fac*sqsum)
    else:
        ave = np.mean(bins, axis=Jax,dtype=bins.dtype)

        err = np.zeros(tL)
        for i in range(tL):
#            sqsum = sum(map(lambda x:x*x,ave[i]-bins[:,i].real))
            sqsum = sum(map(lambda x:x*x,ave[i]-bins[:,i]))
            fac = (Nbins -1) / float(Nbins)
            err[i] = np.sqrt(fac*sqsum.real)        
            
    return (ave,err)
#-------------------------------------------------------

def jackknife_binning(corr, binsize=1):
    
    Jax=0
    
    # Define the number of bins
    mod   = Ncfg%binsize
    Nbins = int((Ncfg - mod) / binsize)
    bins = np.zeros((Nbins,Nt),dtype=corr.dtype)

    csum = np.sum(corr,axis=Jax) # Sum w.r.t to the configurations
    for t in range(Nt):
        for m in np.arange(1,mod+1):
            csum[t] -= corr[Ncfg-m,t]  # Throw away data in case there is modulo
            
        for b in range(Nbins):
            bsum = 0
            for k in range(binsize):
                bsum += corr[b*binsize+k,t]
                
            bins[b][t] = (csum[t] - bsum) / float(Ncfg - binsize - mod) # Bin averages for each bin,t

    return bins, Nbins
#-------------------------------------------------------

print('Done\n')

### Fit functions

In [ ]:
# Constant fit
def constFit(x,cVal):
    return cVal

# Chi-square for constant fit
def chiSq_const(x, y, err, cFit):
    return sum( map(lambda t:t*t, (y-constFit(x,cFit))/err) )
#------------------------

# Linear fit
def linearFit(x,M,b):
    return b + M*x

# Chi-square for Linear fit
def chiSq_linear(x, y, err, M, b):    
    Nd = len(x)
    chi = 0
    for i in range(Nd):
        chi += ( (y[i] - linearFit(x[i],M,b))/err[i] )**2
    
    return chi
#------------------------


# 1 exp. fit of regular summed-ratio
def exp1Fit(x,M,Z,A,dE):
    return Z + M*x + A*x*np.exp(-dE*x)

def chiSq_exp1(x, y, err, M, Z, A, dE):    
    Nd = len(x)
    chi = 0
    for i in range(Nd):
        chi += ( (y[i] - exp1Fit(x[i],M,Z,A,dE))/err[i] )**2
    
    return chi
#------------------------


# 1 exp. fit of reduced-summed ratio
def exp1rFit(x,M,A,dE):
    return M + A*np.exp(-dE*x)

def chiSq_exp1r(x, y, err, M, A, dE):    
    Nd = len(x)
    chi = 0
    for i in range(Nd):
        chi += ( (y[i] - exp1rFit(x[i],M,A,dE))/err[i] )**2
    
    return chi
#------------------------

# 2 exp. fit of reduced-summed ratio
def exp2rFit(x,M,A,B,dE):
    return M + A*np.exp(-dE*x) + B*x*np.exp(-dE*x)

def chiSq_exp2r(x, y, err, M, A, B, dE):    
    Nd = len(x)
    chi = 0
    for i in range(Nd):
        chi += ( (y[i] - exp2rFit(x[i],M,A,B,dE))/err[i] )**2
    
    return chi
#------------------------


# Two-state fit (Original, no factorization)
# def twoStateFit(T,A00,A01,A11,c0,c1,E0,E1):
#     ti,ts = T
    
#     return ( (A00*np.exp(-E0*ts) + A01*np.exp(-E0*(ts-ti))*np.exp(-E1*ti) +
#               A11*np.exp(-E1*ts) + A01*np.exp(-E1*(ts-ti))*np.exp(-E0*ti) ) / 
#                  (c0*np.exp(-E0*ts) + c1*np.exp(-E1*ts)) )


# def chiSq_twoState(T,y,err,A00,A01,A11,c0,c1,E0,E1):
#     chi = 0
#     TI,TS = T
#     Nd = len(TI)
#     i = 0
#     for i in range(Nd):
#         chi += ( (y[i] - twoStateFit((TI[i],TS[i]),A00,A01,A11,c0,c1,E0,E1))/err[i] )**2
#         i += 1
#     return chi
#------------------------


# Two-state fit (Factorized exp. terms)
def twoStateFit(T,A00,A01,A11,c0,c1,dE):
    ti,ts = T
    
    return ( (A00 + 3.0*A11*np.exp(-dE*ts) + A01*(np.exp(-dE*ti)+np.exp(-dE*(ts-ti))) ) / 
                 (2.0*c0 + c1*np.exp(-dE*ts)) )


def chiSq_twoState(T,y,err,A00,A01,A11,c0,c1,dE):
    chi = 0
    TI,TS = T
    Nd = len(TI)
    i = 0
    for i in range(Nd):
        chi += ( (y[i] - twoStateFit((TI[i],TS[i]),A00,A01,A11,c0,c1,dE))/err[i] )**2
        i += 1
    return chi
#------------------------


# Two-state fit (Factorized exp. terms, absorbed c0)
def twoStateFitOpt(T,M,A01,A11,c1,dE):
    ti,ts = T
    
    return ( (M + A11*np.exp(-dE*ts) + A01*(np.exp(-dE*ti)+np.exp(-dE*(ts-ti))) ) / 
                 (1 + c1*np.exp(-dE*ts)) )


def chiSq_twoStateOpt(T,y,err,M,A01,A11,c1,dE):
    chi = 0
    TI,TS = T
    Nd = len(TI)
    i = 0
    for i in range(Nd):
        chi += ( (y[i] - twoStateFitOpt((TI[i],TS[i]),M,A01,A11,c1,dE))/err[i] )**2
        i += 1
    return chi
#------------------------




    
print('Done\n')

## Read two-point functions, perform Jackknife sampling

In [ ]:
def get2ptFileName(phT,t0Tag,src,snk,srow,mFTag):
    filePre = 'corr_2pt.baryon.n64'
    
    srcTag = 'src_%s_%d'%(src,srow)
    snkTag = 'snk_%s_%d'%(snk,srow)
    
    FileName = '%s.%s.%s.%s.%s.%s.dat'%(filePre,phT,t0Tag,srcTag,snkTag,mFTag)
    
    return FileName
#--------------------------

def get2ptFileDir(mainDir,t0Tag,mFTag):
    return '%s/%s/%s' % (mainDir,t0Tag,mFTag)
#--------------------------------------------------------------


c2pt_Allbins   = {}
c2pt_Allave    = {}
c2pt_rowAvg    = {}
c2pt_rawAllave = {}
c2pt_bins      = {}
c2pt_ave       = {}
c2pt_rawAve    = {}

Nbins_2pt_Dict = {}

effEnergy_Allbins = {}
effEnergy_Allave  = {}
effEnergy_bins    = {}
effEnergy_ave     = {}


for opT in SrcSnkOpTypes:
    for phT in phaseTypes:
        momTags = momTags_Dict[(phT,opT)]
                
        for mTag in momTags:
            mVec = momVector_Dict[mTag]
            zMom = mVec[2]
            mFTag = momFileTag(mVec)
            pmTpl = phMomTpl(phT,zMom)

            nRow = nRow2pt_Dict[opT][pmTpl]

            srcOpList2pt = srcOpList2pt_Dict[opT][pmTpl]        
            snkOpList2pt = srcOpList2pt_Dict[opT][pmTpl]

            dTpl = (phT,opT,mTag)
            Ncfg = Ncfg_Dict_2pt[dTpl]

            Nt = Nt_Dict_2pt[pmTpl]

            # Determine number of t0 we consider
            Nt0 = 0
            for t0 in T0List[(phT,opT)]:
                dTpl2pt = (phT,opT,mTag,t0)
                if existsDataSet_2pt[dTpl2pt]:
                    Nt0 += 1

            for isrc, src in enumerate(srcOpList2pt):
                for isnk, snk in enumerate(snkOpList2pt):
                    ckey = (opT,phT,mTag,isrc,isnk) # All info in one tuple

                    c2pt_rawAll = np.zeros((Ncfg,Nt,nRow,Nt0),dtype=np.complex128)

                    for it0,t0 in enumerate(T0List[(phT,opT)]):
                        dTpl2pt = (phTS,opTS,mTag,t0)
                        if existsDataSet_2pt[dTpl2pt]:
                            t0Tag = 't0_%d'%(t0)

                            for ir,irow in enumerate(range(1,nRow+1)):
                                ckeyA = (opT,phT,mTag,isrc,isnk,t0,irow) # All info in one tuple

                                c2ptFileDir  = get2ptFileDir(corr2ptMainDir[(phT,opT)],t0Tag,mFTag)
                                c2ptFileName = get2ptFileName(phT,t0Tag,src,snk,irow,mFTag)               
                                c2ptFile = '%s/%s' % (c2ptFileDir,c2ptFileName)

                                with open(c2ptFile) as fp:
                                    line = fp.readlines()
                                    c = 0
                                    for n in line:
                                        it   = c%Nt
                                        icfg = c//Nt
                                        c2pt_rawAll[icfg,it,ir,it0] = complex(np.float64(n.split()[1]),
                                                                              np.float64(n.split()[2]))
                                        c += 1
                                # Reading file

                                # Jackknife average on All two-point functions
                                c2pt_Allbins[ckeyA], Nbins_2pt_Dict[dTpl] = jackknife_binning(c2pt_rawAll[:,:,ir,it0].real)
                                Nbins_2pt = Nbins_2pt_Dict[dTpl]
                                c2pt_Allave[ckeyA] = jackknife_ave(c2pt_Allbins[ckeyA],Nbins_2pt,tL=Nt)

                                # Effective energy of All t0 and rows
                                effEnergy_Allbins[ckeyA] = np.zeros((Nbins_2pt,Nt))
                                for it in range(Nt):
                                    effEnergy_Allbins[ckeyA][:,it] = np.log(c2pt_Allbins[ckeyA][:,it]/
                                                                            c2pt_Allbins[ckeyA][:,(it+1)%Nt])

                                effEnergy_Allave[ckeyA] = jackknife_ave(effEnergy_Allbins[ckeyA],Nbins_2pt,Nt)                                
                            # for-irow

                            # Standard average and standard error (still average over rows)
                            # Might need this for constructing covariance matrix
                            ckeyB = (opT,phT,mTag,isrc,isnk,t0)
                            c2pt_rowAvg[ckeyB] = np.mean(c2pt_rawAll[:,:,:,it0],axis=2,dtype=np.complex128) # Average over rows
                            c2pt_rawAllave[ckeyB] = (np.mean(c2pt_rowAvg[ckeyB],axis=0),
                                                        np.std(c2pt_rowAvg[ckeyB],axis=0)/np.sqrt(Ncfg))
                        else:
                            print('### Skipping dataset:', dTpl2pt)
                    # for-t0

                    # Average over rows and t0
                    c2pt_Tmp1 = np.mean(c2pt_rawAll,axis=3,dtype=np.complex128) # over t0
                    c2pt_Tmp2 = np.mean(c2pt_Tmp1  ,axis=2,dtype=np.complex128) # over rows

                    # Standard average and standard error
                    c2pt_rawAve[ckey] = (np.mean(c2pt_Tmp2,axis=0),np.std(c2pt_Tmp2,axis=0)/np.sqrt(Ncfg))

                    # Jackknife-average on t0- and row-average            
                    c2pt_bins[ckey], Nbins_2pt = jackknife_binning(c2pt_Tmp2.real)
                    c2pt_ave[ckey] = jackknife_ave(c2pt_bins[ckey],Nbins_2pt,tL=Nt)

                    effEnergy_bins[ckey] = np.zeros((Nbins_2pt,Nt))
                    for it in range(Nt):
                        effEnergy_bins[ckey][:,it] = np.log(c2pt_bins[ckey][:,it]/
                                                            c2pt_bins[ckey][:,(it+1)%Nt])

                    effEnergy_ave[ckey] = jackknife_ave(effEnergy_bins[ckey],Nbins_2pt,tL=Nt)                    
                    
            # for-src-snk
            print('Done: Momentum %s'%(mTag))
        # for momentum
        
           
print('\nDone\n')

## Perform constant fit on the Effective Energy

In [ ]:

tini = 7
tfin = 13
tFit = np.arange(tini,tfin+1)
Ndata = len(tFit)

E0_fit = {}
E0_cov = {}

E0_fit_ave = {}
for opT in SrcSnkOpTypes:
    for phT in phaseTypes:
        momTags = momTags_Dict[(phT,opT)]
                
        for mTag in momTags:
            adTplE = (phT,opT,mTag)
            if analyzeDataSetE[adTplE]:
                mVec = momVector_Dict[mTag]
                zMom = mVec[2]
                pmTpl = phMomTpl(phT,zMom)

                srcOpList2pt = srcOpList2pt_Dict[opT][pmTpl]        
                snkOpList2pt = srcOpList2pt_Dict[opT][pmTpl]

                dTpl = (phT,opT,mTag)
                Nbins = Nbins_2pt_Dict[dTpl]

                for isrc, src in enumerate(srcOpList2pt):
                    for isnk, snk in enumerate(snkOpList2pt):
                        ckey = (opT,phT,mTag,isrc,isnk) # All info in one tuple

                        E0_fit[ckey] = np.zeros(Nbins)
                        E0_cov[ckey] = np.zeros((Nbins,Ndata))

                        E0_fitAll = np.zeros(Nbins)
                        E0_covAll = np.zeros((Nbins,Ndata))

                        nFit = 0
                        for ib in range(Nbins):
                            ydata = effEnergy_bins[ckey][ib,tini:tfin+1]
                            yerr  = effEnergy_ave[ckey][1][tini:tfin+1] 
                            if (not np.isnan(ydata).any()):
                                E0_fitAll[nFit], E0_covAll[nFit] = scipyOpt.curve_fit(constFit,tFit,ydata,sigma=yerr)
                                nFit += 1

                        E0_fit[ckey] =  E0_fitAll[:nFit]

                        if (np.shape(E0_fit[ckey])==(0,)) or (np.shape(E0_fit[ckey])==(1,)):
                            E0_fit_ave[ckey] = (0,0)
                        else:
                            E0_fit_ave[ckey] = jackknife_ave(E0_fit[ckey],Nbins=nFit,tL=1)
            else:
                print('Skipping dataset: ',adTplE)

            print('%s Done'%(mTag))
        # for-momentum
            
    
print('\nDone\n')


## Plot Effective energy

In [ ]:
# Two-point function
fig1, ax1 = plt.subplots(figsize=[15,9.4])
fig2, ax2 = plt.subplots(figsize=[15,9.4])
#fig3, ax3 = plt.subplots(figsize=[15,9.4])

elw=1
ecs=6

iSrc = 0 #srcOpList.index('NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1')
iSnk = 0 #snkOpList.index('NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1')
print('isrc: %s\nisnk: %s'%(iSrc,iSnk))

phT = None
opT = None
phTP = 'unphased'
opTP = 'single'

# Plot one momentum, all effective energies
mTag = 'pz.+1'
mVec = momVector_Dict[mTag]
zMom = mVec[2]
pmTpl = phMomTpl(phTP,zMom)

Nt = Nt_Dict_2pt[pmTpl]

runTag = None#'%s_%s'%(phaseInfo,pL)

nRow = nRow2pt_Dict[opTP][pmTpl]
xRng = np.arange(Nt)

i=0
osi=0
for t0 in T0List[(phTP,opTP)]:
    dTpl2pt = (phTP,opTP,mTag,t0)
    if existsDataSet_2pt[dTpl2pt]:

        for irow in range(1,nRow+1):
            ckey = (opTP,phTP,mTag,iSrc,iSnk,t0,irow)

            ax1.errorbar(xRng+osDctF[osi],  effEnergy_Allave[ckey][0].real,
                         yerr = effEnergy_Allave[ckey][1],
                         marker = mDct[i], color = cDct[i], ls='none', label=r'$t_0=%d$, row=%d'%(t0,irow),
                         ms = 8.0, elinewidth=elw, capsize=ecs,fillstyle=fsDct[irow])

            ax1.axhline(dispRel(amN,mVec),lw=1,color='k',ls='--')

            osi+=1
        # for-irow
        i+=1
    else:
        print('Skipping ', dTpl2pt)

ckey = (opTP,phTP,mTag,iSrc,iSnk)
ax1.errorbar(xRng-0.2,  effEnergy_ave[ckey][0].real,
             yerr = effEnergy_ave[ckey][1],
             fmt = 'kX', label=r'$\vec{p}=(%d,%d,%d)$ Avg.'%(mVec[0],mVec[1],mVec[2]),
             ms = 9.0, elinewidth=2, capsize=ecs)#,fillstyle='none')
    
ax1.text(0,0.6,r'$\vec{p}=(%d,%d,%d)$'%(mVec[0],mVec[1],mVec[2]),fontsize=18)    

ax1.set_xlabel(r'$t/a$',fontsize=18)
ax1.set_ylabel(r'$aE_{\rm eff}(t)$',fontsize=18)
ax1.xaxis.set_ticks(np.arange(0, 20, 2))

ax1.set_ylim(0.5,1.2)
ax1.yaxis.set_ticks(np.arange(0.45, 0.86, 0.05))

ax1.legend(loc='best', ncol=2, numpoints=1, fontsize=18,
           labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
           markerscale=0.8)

fig1.savefig('%s/aEeff.%s.all_t0.rowAll.pdf'%(corr2ptWorkDir,momFileTag(mVec)), bbox_inches='tight')

#-------------------------------------------------------------------------------------

# Plot average Eff. energy for all momenta
i=0
for mTag in momTags_Dict[(phTP,opTP)]:
    mVec = momVector_Dict[mTag]
    zMom = mVec[2]
    adTplE = (phTP,opTP,mTag)
    if analyzeDataSetE[adTplE] and (np.abs(zMom)<3):
        mVec = momVector_Dict[mTag]
        zMom = mVec[2]
        pmTpl = phMomTpl(phTP,zMom)

        Nt = Nt_Dict_2pt[pmTpl]

        runTag = None#'%s_%s'%(phaseInfo,pL)
        nRow = nRow2pt_Dict[opTP][pmTpl]
        xRng = np.arange(Nt)

        ckey = (opTP,phTP,mTag,iSrc,iSnk)

        ax2.errorbar(xRng,  effEnergy_ave[ckey][0].real,
                     yerr = effEnergy_ave[ckey][1],
                     marker = mDct[i], color = cDct[i], ls='none',
                     label=r'$\vec{p}=(%d,%d,%d)$'%(mVec[0],mVec[1],mVec[2]),
                     ms = 8.0, elinewidth=elw, capsize=ecs)

        ax2.axhline(dispRel(amN,mVec),lw=1,color=cDct[i],ls='--')

        cFitv = E0_fit_ave[ckey][0]
        cFite = E0_fit_ave[ckey][1]
        if cFitv != 0 and cFite != 0:
            ax2.fill_between(tFit, cFitv-cFite, cFitv+cFite, facecolor=cDct[i],
                             alpha=0.4)#, label= 'const. fit')
        else:
            print('Fit for ', ckey, 'was not proper')

        i+=1
    
ax2.set_xlabel(r'$t/a$',fontsize=18)
ax2.set_ylabel(r'$aE_{\rm eff}(t)$',fontsize=18)
ax2.xaxis.set_ticks(np.arange(0, 22, 2))

ax2.set_ylim(0.45,0.85)
ax2.yaxis.set_ticks(np.arange(0.45, 0.86, 0.05))

ax2.legend(loc='best', ncol=1, numpoints=1, fontsize=18,
           labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
           markerscale=0.8)

fig2.savefig('%s/aEeff.allMom.t0Ave.rowAve.pdf'%(corr2ptWorkDir),bbox_inches='tight')

#-------------------------------------------------------------------------------------


### Average two-point functions over momentum

In [ ]:

c2ptA_bins      = {}
c2ptA_ave       = {}
c2pt_rawAve    = {}


for opT in SrcSnkOpTypes:
    for phT in phaseTypes:
        momTags = momTags_Dict_ave[(phT,opT)]
                
        for mTag in momTags:
            mVec = momVector_Dict[mTag]
            zMom = mVec[2]
            mFTag = momFileTag(mVec)
            pmTpl = phMomTpl(phT,zMom)

            nRow = nRow2pt_Dict[opT][pmTpl]

            srcOpList2pt = srcOpList2pt_Dict[opT][pmTpl]        
            snkOpList2pt = srcOpList2pt_Dict[opT][pmTpl]

            dTpl = (phT,opT,mTag)
            Nbins =  Nbins_2pt_Dict[dTpl]
            
            Nt = Nt_Dict_2pt[pmTpl]

            for isrc, src in enumerate(srcOpList2pt):
                for isnk, snk in enumerate(snkOpList2pt):
                    ckey = (opT,phT,mTag,isrc,isnk) # All info in one tuple

                    c2ptA_bins[ckey] = np.zeros((Nbins,Nt))
                    if mTag == 'pz.0':
                        c2ptA_bins[ckey] = c2pt_bins[ckey]
                    else:
                        mTagN = 'pz.-%d'%(mVec[2])

                        ckeyP = (opT,phT,mTag ,isrc,isnk)
                        ckeyN = (opT,phT,mTagN,isrc,isnk)

                        c2ptA_bins[ckey] = 0.5 * (c2pt_bins[ckeyP] + c2pt_bins[ckeyN])
                    
                    c2ptA_ave[ckey] = jackknife_ave(c2ptA_bins[ckey],Nbins,tL=Nt)
            # for-src-snk
            
            print('Done: Momentum %s'%(mTag))
        # for momentum
        
           
print('\nDone\n')

## Read / Load three-point functions, perform Jackknife sampling, average over $p_z$ and $z_3$

In [ ]:
c3pt_data_Source  = {'Read': False, 'Dump': False, 'Load': False}


In [ ]:

if c3pt_data_Source['Read']:
    def get3ptFileName(phT,t0Tag,tsnkTag,src,snk,srcRow,snkRow,ins,insRow,mFTag,d):
        filePre = 'corr_3pt.baryon.n64'

        srcTag = 'src_%s_%d'%(src,srcRow)
        snkTag = 'snk_%s_%d'%(snk,snkRow)
        insTag = 'ins_%s_%d'%(ins,insRow)

        dTag = dispTag(d)

        FileName = '%s.%s.%s.%s.%s.%s.%s.%s.%s.dat'%(filePre,phT,t0Tag,tsnkTag,srcTag,snkTag,insTag,dTag,mFTag)

        return FileName
    #--------------------------

    def get3ptFileDir(mainDir,t0Tag,mFTag,tsnkTag):
        return '%s/%s/%s/%s' % (mainDir,t0Tag,mFTag,tsnkTag)
    #--------------------------------------------------------------


    c3pt_Allbins_Re = {}
    c3pt_Allbins_Im = {}
    c3pt_Allave_Re  = {}
    c3pt_Allave_Im  = {}

    c3pt_rawAll = {}

    for opT in SrcSnkOpTypes:
        for phT in phaseTypes:
            momTags = momTags_Dict[(phT,opT)]

            for mTag in momTags:
                mVec = momVector_Dict[mTag]
                zMom = mVec[2]
                mFTag = momFileTag(mVec)
                pmTpl = phMomTpl(phT,zMom)

                nRow = nRow3pt_Dict[opT][pmTpl]

                srcOpList3pt = srcOpList3pt_Dict[opT][pmTpl]        
                snkOpList3pt = srcOpList3pt_Dict[opT][pmTpl]        

                for tsnk in TsinkList[(phT,opT)]:
                    tsnkTag = 'tsnk_%d'%(tsnk)
                    dTpl = (phT,opT,mTag,tsnk)
                    Ncfg = Ncfg_Dict_3pt[dTpl]
                    Nt = tsnk

                    for it0,t0 in enumerate(T0List[(phT,opT)]):
                        t0Tag = 't0_%d'%(t0)

                        dTpl3pt = (phT,opT,mTag,tsnk,t0)
                        dTpl2pt = (phTS,opTS,mTag,t0)
                        if existsDataSet_3pt[dTpl3pt] and existsDataSet_2pt[dTpl2pt]:
                            print('Reading dataset: ', dTpl3pt)
                            for isrc, src in enumerate(srcOpList3pt):
                                for isnk, snk in enumerate(snkOpList3pt):
                                    for gMat in gMatList:
                                        insOpName = gMatMap[phT][gMat][0]
                                        insOpRow  = gMatMap[phT][gMat][1]

                                        for irowSrc in range(1,nRow+1):
                                            for irowSnk in range(1,nRow+1):

                                                for d in dispLen:
                                                    ckeyA = (opT,phT,mTag,tsnk,isrc,isnk,gMat,d,irowSrc,irowSnk,t0) # All info in one tuple
                                                    c3pt_rawAll[ckeyA] = np.zeros((Ncfg,Nt),dtype=np.complex128)

                                                    c3ptFileDir  = get3ptFileDir(corr3ptMainDir[(phT,opT)],t0Tag,mFTag,tsnkTag)
                                                    c3ptFileName = get3ptFileName(phT,t0Tag,tsnkTag,src,snk,irowSrc,irowSnk,
                                                                                  insOpName,insOpRow,mFTag,d)               
                                                    c3ptFile = '%s/%s' % (c3ptFileDir,c3ptFileName)

                                                    with open(c3ptFile) as fp:
                                                        line = fp.readlines()
                                                        c = 0
                                                        for n in line:
                                                            it   = c%Nt
                                                            icfg = c//Nt
                                                            c3pt_rawAll[ckeyA][icfg,it] = complex(np.float64(n.split()[1]),
                                                                                                  np.float64(n.split()[2]))
                                                            c += 1
                                                    # Reading file

                                                    # Jackknife average on All three-point functions
                                                    c3pt_Allbins_Re[ckeyA], Nbins_tmp1 = jackknife_binning(c3pt_rawAll[ckeyA].real)
                                                    c3pt_Allbins_Im[ckeyA], Nbins_tmp2 = jackknife_binning(c3pt_rawAll[ckeyA].imag)
                                                    c3pt_Allave_Re[ckeyA] = jackknife_ave(c3pt_Allbins_Re[ckeyA],Nbins_tmp1,tL=Nt)
                                                    c3pt_Allave_Im[ckeyA] = jackknife_ave(c3pt_Allbins_Im[ckeyA],Nbins_tmp2,tL=Nt)
                                                # for-displacement
                                        # for-Rows
                            # for-src-snk-ins
                        else:
                            print('### Skipping dataset:', dTpl3pt)
                    # for-t0

                    print('Done: Momentum %s , t_sink = %d'%(mTag,tsnk))
                # for tsnk
            # for momentum
else:
    print('Skipping reading the raw three-point functions')
            
print('\nDone\n')

### Dump / Load raw three-point function using Pickle

In [ ]:
if c3pt_data_Source['Dump']:
    print('Dumping the raw three-point functions...')
    pickle.dump(c3pt_rawAll    , open( 'c3pt_rawAll.p'    , 'wb' ))
# pickle.dump(c3pt_Allbins_Re, open( 'c3pt_Allbins_Re.p', 'wb' ))
# pickle.dump(c3pt_Allbins_Im, open( 'c3pt_Allbins_Im.p', 'wb' ))
# pickle.dump(c3pt_Allave_Re , open( 'c3pt_Allave_Re.p' , 'wb' ))
# pickle.dump(c3pt_Allave_Im , open( 'c3pt_Allave_Im.p' , 'wb' ))
elif c3pt_data_Source['Load']:
    print('Loading the raw three-point functions...')
    c3pt_rawAll = pickle.load( open( "c3pt_rawAll.p", "rb" ) )
else:
    print('Skipping dumping / loading of raw three-point functions')
    
print('\nDone\n')

### Average three-point functions over $t_0$, perform Jackknife sampling

In [ ]:
c3pt_sampling = {'Evaluate': False, 'Dump': False, 'Load': False}


In [ ]:

if c3pt_sampling['Evaluate']:

    c3pt_rawAve = {}

    c3pt_bins_Re = {}
    c3pt_bins_Im = {}

    c3pt_ave_Re = {}
    c3pt_ave_Im = {}

    Nbins_3pt_Dict = {}

    for opT in SrcSnkOpTypes:
        for phT in phaseTypes:
            momTags = momTags_Dict[(phT,opT)]

            for mTag in momTags:
                mVec = momVector_Dict[mTag]
                zMom = mVec[2]
                mFTag = momFileTag(mVec)
                pmTpl = phMomTpl(phT,zMom)

                nRow = nRow3pt_Dict[opT][pmTpl]

                srcOpList3pt = srcOpList3pt_Dict[opT][pmTpl]        
                snkOpList3pt = srcOpList3pt_Dict[opT][pmTpl]        

                for tsnk in TsinkList[(phT,opT)]:
                    tsnkTag = 'tsnk_%d'%(tsnk)
                    dTpl = (phT,opT,mTag,tsnk)
                    Ncfg = Ncfg_Dict_3pt[dTpl]
                    Nt = tsnk

                    adTplM = (phT,opT,mTag,tsnk)
                    if analyzeDataSetM[adTplM]:                
                        for isrc, src in enumerate(srcOpList3pt):
                            for isnk, snk in enumerate(snkOpList3pt):
                                for gMat in gMatList:
                                    insOpName = gMatMap[phT][gMat][0]
                                    insOpRow  = gMatMap[phT][gMat][1]

                                    for irowSrc in range(1,nRow+1):
                                        for irowSnk in range(1,nRow+1):

                                            for d in dispLen:
                                                ckey = (opT,phT,mTag,tsnk,isrc,isnk,gMat,d,irowSrc,irowSnk) # All info in one tuple


                                                # Average over t0
                                                c3pt_t0_ave = np.zeros((Ncfg,Nt),dtype=np.complex128)
                                                Nt0 = 0
                                                for it0,t0 in enumerate(T0List[(phT,opT)]):
                                                    ckeyA = (opT,phT,mTag,tsnk,isrc,isnk,gMat,d,irowSrc,irowSnk,t0) # All info in one tuple

                                                    dTpl3pt = (phT,opT,mTag,tsnk,t0)
                                                    dTpl2pt = (phTS,opTS,mTag,t0)
                                                    if existsDataSet_3pt[dTpl3pt] and existsDataSet_2pt[dTpl2pt]:
                                                        Nt0 += 1
                                                        c3pt_t0_ave = c3pt_t0_ave + c3pt_rawAll[ckeyA]                                        
                                                # for-t0
                                                c3pt_t0_ave = c3pt_t0_ave / float(Nt0)


                                                # Standard average and standard error
                                                c3pt_rawAve[ckey] = (np.mean(c3pt_t0_ave,axis=0,dtype=np.complex128),
                                                                     np.std(c3pt_t0_ave,axis=0,dtype=np.complex128)/np.sqrt(Ncfg))

                                                # Jackknife-average on t0- and row-average            
                                                c3pt_bins_Re[ckey], Nbins_3pt_Dict[dTpl] = jackknife_binning(c3pt_t0_ave.real)
                                                c3pt_bins_Im[ckey], Nbins_3pt_tmp3       = jackknife_binning(c3pt_t0_ave.imag)

                                                c3pt_ave_Re[ckey] = jackknife_ave(c3pt_bins_Re[ckey],Nbins_3pt_Dict[dTpl],tL=Nt)
                                                c3pt_ave_Im[ckey] = jackknife_ave(c3pt_bins_Im[ckey],Nbins_3pt_Dict[dTpl],tL=Nt)

                                        # for-displacement
                        # for-src-snk-ins
                        print('Done: Momentum %s , t_sink = %d'%(mTag,tsnk))
                    # if-analyze
                # for tsnk
            # for momentum
else:
    print('Skipping evaluating the three-point function samples')
        
           
print('\nDone\n')

### Write / Load non-averaged 3pt-function samples and average

In [ ]:
if c3pt_sampling['Dump']:
    print('Dumping the three-point function samples...')
    pickle.dump(c3pt_rawAve, open('c3pt_rawAve.p', 'wb' ))

    pickle.dump(c3pt_bins_Re, open('c3pt_bins_Re.p', 'wb' ))
    pickle.dump(c3pt_bins_Im, open('c3pt_bins_Im.p', 'wb' ))

    pickle.dump(c3pt_ave_Re, open('c3pt_ave_Re.p', 'wb' ))
    pickle.dump(c3pt_ave_Im, open('c3pt_ave_Im.p', 'wb' ))

    pickle.dump(Nbins_3pt_Dict, open('Nbins_3pt_Dict.p', 'wb' ))
elif c3pt_sampling['Load']:
    print('Loading the three-point function samples...')
#    c3pt_rawAve = pickle.load( open('c3pt_rawAve.p', 'rb'))
    
    c3pt_bins_Re = pickle.load( open('c3pt_bins_Re.p', 'rb'))
    c3pt_bins_Im = pickle.load( open('c3pt_bins_Im.p', 'rb'))

    c3pt_ave_Re = pickle.load( open('c3pt_ave_Re.p', 'rb'))
    c3pt_ave_Im = pickle.load( open('c3pt_ave_Im.p', 'rb'))

    Nbins_3pt_Dict = pickle.load( open('Nbins_3pt_Dict.p', 'rb'))
else:
    print('Skipping dumping/loading of three-point function samples')

print('\nDone\n')

### Average three-point function over $p_z$ and $z$ values

In [ ]:
c3pt_ave_sampling = {'Evaluate': False, 'Dump': False, 'Load': False}

c3ptA_bins = {'Re': {}, 'Im': {}}
c3ptA_ave  = {'Re': {}, 'Im': {}}

In [ ]:

if c3pt_ave_sampling['Evaluate']:

    for opT in SrcSnkOpTypes:
        for phT in phaseTypes:
            momTags = momTags_Dict_ave[(phT,opT)]

            for mTag in momTags:
                mVec = momVector_Dict[mTag]
                zMom = mVec[2]
                mFTag = momFileTag(mVec)
                pmTpl = phMomTpl(phT,zMom)

                nRow = nRow3pt_Dict[opT][pmTpl]

                srcOpList3pt = srcOpList3pt_Dict[opT][pmTpl]        
                snkOpList3pt = srcOpList3pt_Dict[opT][pmTpl]        

                for tsnk in TsinkList[(phT,opT)]:
                    tsnkTag = 'tsnk_%d'%(tsnk)
                    dTpl = (phT,opT,mTag,tsnk)
                    Nbins = Nbins_3pt_Dict[dTpl]
                    Nt = tsnk

                    for isrc, src in enumerate(srcOpList3pt):
                        for isnk, snk in enumerate(snkOpList3pt):
                            for gMat in gMatList:
                                insOpName = gMatMap[phT][gMat][0]
                                insOpRow  = gMatMap[phT][gMat][1]

                                for irowSrc in range(1,nRow+1):
                                    for irowSnk in range(1,nRow+1):

                                        for d in dispLen_ave:
                                            ckey = (opT,phT,mTag,tsnk,isrc,isnk,gMat,d,irowSrc,irowSnk)

                                            c3ptA_bins['Re'][ckey] = np.zeros((Nbins,Nt))
                                            c3ptA_bins['Im'][ckey] = np.zeros((Nbins,Nt))

                                            if mTag == 'pz.0':
                                                adTplM = (phT,opT,mTag,tsnk)
                                                if analyzeDataSetM[adTplM]:
                                                    if d==0: # (pz=0, z=0)
                                                        c3ptA_bins['Re'][ckey] = c3pt_bins_Re[ckey]
                                                        c3ptA_bins['Im'][ckey] = c3pt_bins_Im[ckey]
                                                    else: # (pz=0, z/=0)
                                                        ckey_Posd = (opT,phT,mTag,tsnk,isrc,isnk,gMat, d,irowSrc,irowSnk)
                                                        ckey_Negd = (opT,phT,mTag,tsnk,isrc,isnk,gMat,-d,irowSrc,irowSnk)
                                                        c3ptA_bins['Re'][ckey] = 0.5 * (c3pt_bins_Re[ckey_Posd] + c3pt_bins_Re[ckey_Negd])
                                                        c3ptA_bins['Im'][ckey] = 0.5 * (c3pt_bins_Re[ckey_Posd] - c3pt_bins_Re[ckey_Negd])
                                            else:
                                                mTagN = 'pz.-%d'%(mVec[2])

                                                adTplM = (phT,opT,mTag,tsnk)
                                                if analyzeDataSetM[adTplM]:
                                                    if d==0: # (pz/=0, z=0)
                                                        ckey_PosP = (opT,phT,mTag ,tsnk,isrc,isnk,gMat,d,irowSrc,irowSnk)
                                                        ckey_NegP = (opT,phT,mTagN,tsnk,isrc,isnk,gMat,d,irowSrc,irowSnk)
                                                        c3ptA_bins['Re'][ckey] = 0.5 * (c3pt_bins_Re[ckey_PosP] + c3pt_bins_Re[ckey_NegP])
                                                        c3ptA_bins['Im'][ckey] = 0.5 * (c3pt_bins_Re[ckey_PosP] - c3pt_bins_Re[ckey_NegP])
                                                    else: # (pz/=0, z/=0)
                                                        ckey_PosP_Posd = (opT,phT,mTag ,tsnk,isrc,isnk,gMat, d,irowSrc,irowSnk)
                                                        ckey_PosP_Negd = (opT,phT,mTag ,tsnk,isrc,isnk,gMat,-d,irowSrc,irowSnk)
                                                        ckey_NegP_Posd = (opT,phT,mTagN,tsnk,isrc,isnk,gMat, d,irowSrc,irowSnk)
                                                        ckey_NegP_Negd = (opT,phT,mTagN,tsnk,isrc,isnk,gMat,-d,irowSrc,irowSnk)

                                                        c3ptA_bins['Re'][ckey] = 0.25 * (c3pt_bins_Re[ckey_PosP_Posd] + c3pt_bins_Re[ckey_PosP_Negd] +
                                                                                         c3pt_bins_Re[ckey_NegP_Posd] + c3pt_bins_Re[ckey_NegP_Negd])

                                                        c3ptA_bins['Im'][ckey] = 0.25 * (c3pt_bins_Im[ckey_PosP_Posd] - c3pt_bins_Im[ckey_PosP_Negd] -
                                                                                         c3pt_bins_Im[ckey_NegP_Posd] + c3pt_bins_Im[ckey_NegP_Negd])
                                            # End-if

                                            # Jackknife-average on p,z-averaged three-point function
                                            for ri in ReIm:
                                                c3ptA_ave[ri][ckey] = jackknife_ave(c3ptA_bins[ri][ckey],Nbins_3pt_Dict[dTpl],tL=Nt)
                                        # for-displacement
                    # for-src-snk-ins
                    print('Done: Momentum %s , t_sink = %d'%(mTag,tsnk))
                # for tsnk
            # for momentum
else:
    print('Skipping evaluating the averaged three-point function samples')


print('\nDone\n')


### Write / Load 3pt-function p,z-averaged bins and average

In [ ]:
if c3pt_ave_sampling['Dump']:
    print('Dumping the averaged three-point function samples...')
    for ri in ReIm:
        pickle.dump(c3ptA_bins[ri], open('c3ptA_bins_%s.p'%(ri), 'wb' ))
        pickle.dump(c3ptA_ave[ri],  open('c3ptA_ave_%s.p'%(ri), 'wb' ))

    pickle.dump(Nbins_3pt_Dict, open('Nbins_3pt_Dict.p', 'wb' ))
elif c3pt_ave_sampling['Load']:
    print('Loading the averaged three-point function samples...')
    for ri in ReIm:
        c3ptA_bins[ri] = pickle.load(open('c3ptA_bins_%s.p'%(ri), 'rb'))
        c3ptA_ave[ri]  = pickle.load(open('c3ptA_ave_%s.p'%(ri), 'rb'))

    Nbins_3pt_Dict = pickle.load( open('Nbins_3pt_Dict.p', 'rb'))
else:
    print('Skipping dumping/loading of averaged three-point function samples')



print('\nDone\n')

## Construct Ratio of Averaged three- and two-point functions, $R(t_s,t_{\rm ins}) = C_{3pt}(t_s,t_{\rm ins})\; /\; C_{2pt}(t_s)$, and perform Summation of Ratio


In [ ]:
ratio_Source = {'Evaluate': False, 'Dump': False, 'Load': True}

RA_bins = {}
RA_ave  = {}

RsumA_bins = {}
RsumA_ave  = {}

RrsumA_bins = {}
RrsumA_ave  = {}



In [ ]:
if ratio_Source['Evaluate']:

    for ri in ReIm:
        RA_bins[ri] = {}
        RA_ave[ri] = {}

        RsumA_bins[ri] = {}
        RsumA_ave[ri] = {}

        RrsumA_bins[ri] = {}
        RrsumA_ave[ri] = {}

        
    for opT in SrcSnkOpTypes:
        for phT in phaseTypes:
            momTags = momTags_Dict_ave[(phT,opT)]

            for mTag in momTags:
                mVec = momVector_Dict[mTag]
                zMom = mVec[2]
                mFTag = momFileTag(mVec)
                pmTpl = phMomTpl(phT,zMom)

                srcOpList = srcOpList3pt_Dict[opT][pmTpl] # These are the same in two- and
                snkOpList = srcOpList3pt_Dict[opT][pmTpl] # three-point functions

                for isrc, src in enumerate(srcOpList):
                    for isnk, snk in enumerate(snkOpList):
                        ckey2 = (opT,phT,mTag,isrc,isnk) # 2pt-tuple

                        for tsnk in TsinkList[(phT,opT)]:
                            tsnkTag = 'tsnk_%d'%(tsnk)
                            Ntins = tsnk

                            dTpl = (phT,opT,mTag,tsnk)
                            Nbins = Nbins_3pt_Dict[dTpl]

                            adTplM = (phT,opT,mTag,tsnk)
                            if analyzeDataSetM[adTplM]:
                                for irowSrc in range(1,nRow+1):
                                    for irowSnk in range(1,nRow+1):                        
                                        for gMat in gMatList:
                                            insOpName = gMatMap[phT][gMat][0]
                                            insOpRow  = gMatMap[phT][gMat][1]

                                            for z in dispLen_ave:
                                                ckey3 = (opT,phT,mTag,tsnk,isrc,isnk,gMat,z,irowSrc,irowSnk) # 3pt-tuple

                                                for ri in ReIm:
                                                    RA_bins[ri][ckey3] = np.zeros((Nbins,Ntins))
                                                    RsumA_bins[ri][ckey3] = np.zeros(Nbins)
                                                    for tins in range(Ntins):
                                                        RA_bins[ri][ckey3][:,tins] = c3ptA_bins[ri][ckey3][:,tins] / c2ptA_bins[ckey2][:,tsnk]

                                                        if tins>0: # Exclude source contact term
                                                            RsumA_bins[ri][ckey3] += RA_bins[ri][ckey3][:,tins]
                                                            
                                                    RA_ave[ri][ckey3] = jackknife_ave(RA_bins[ri][ckey3],Nbins,tL=Ntins)
                                                    RsumA_ave[ri][ckey3] = jackknife_ave(RsumA_bins[ri][ckey3],Nbins,tL=1)
                        # for-tsink

                        # Evaluate reduced summed ratio
#                        TsinkList_rs = {'l': TsinkList[(phT,opT)][:-1],'h': TsinkList[(phT,opT)][1:]}
                        TsinkList_rs = TsinkList[(phT,opT)][:-1]
                        for its,tsnk in enumerate(TsinkList_rs):
                            tsnkH = TsinkList[(phT,opT)][its+1]
                            tsnkL = TsinkList[(phT,opT)][its]

                            dTpl = (phT,opT,mTag,tsnk)
                            Nbins = Nbins_3pt_Dict[dTpl]

                            adTplM = (phT,opT,mTag,tsnk)
                            if analyzeDataSetM[adTplM]:
                                for irowSrc in range(1,nRow+1):
                                    for irowSnk in range(1,nRow+1):                        
                                        for gMat in gMatList:

                                            for z in dispLen_ave:
                                                ckey3  = (opT,phT,mTag,tsnk ,isrc,isnk,gMat,z,irowSrc,irowSnk)
                                                ckey3H = (opT,phT,mTag,tsnkH,isrc,isnk,gMat,z,irowSrc,irowSnk)
                                                ckey3L = (opT,phT,mTag,tsnkL,isrc,isnk,gMat,z,irowSrc,irowSnk)
                                                for ri in ReIm:
                                                    RrsumA_bins[ri][ckey3] = (RsumA_bins[ri][ckey3H] - RsumA_bins[ri][ckey3L]) / (tsnkH - tsnkL)
                                                    RrsumA_ave[ri][ckey3] = jackknife_ave(RrsumA_bins[ri][ckey3],Nbins,tL=1)
                            # end-if
                        # for-tsink


                print('Done: Momentum %s'%(mTag))
else:
    print('Skipping evaluating the Ratio')


print('\nDone\n')

### Write / Load averaged Ratio of three- and two-point functions, $R(t_s,t_{\rm ins}) = C_{3pt}(t_s,t_{\rm ins})\; /\; C_{2pt}(t_s)$

In [ ]:
if ratio_Source['Dump']:
    print('Dumping the Ratio...')
    pickle.dump(RA_bins, open('RA_bins.p', 'wb' ))
    pickle.dump(RA_ave, open( 'RA_ave.p', 'wb' ))
    pickle.dump(RsumA_bins, open('RsumA_bins.p', 'wb' ))
    pickle.dump(RsumA_ave, open( 'RsumA_ave.p', 'wb' ))
    pickle.dump(RrsumA_bins, open('RrsumA_bins.p', 'wb' ))
    pickle.dump(RrsumA_ave, open( 'RrsumA_ave.p', 'wb' ))
elif ratio_Source['Load']:
    print('Loading the Ratio...')
    RA_bins = pickle.load(open('RA_bins.p', 'rb'))
    RA_ave = pickle.load( open('RA_ave.p', 'rb'))
    RsumA_bins = pickle.load(open('RsumA_bins.p', 'rb'))
    RsumA_ave = pickle.load( open('RsumA_ave.p', 'rb'))
    RrsumA_bins = pickle.load(open('RrsumA_bins.p', 'rb'))
    RrsumA_ave = pickle.load( open('RrsumA_ave.p', 'rb'))

    Nbins_3pt_Dict = pickle.load( open('Nbins_3pt_Dict.p', 'rb'))
else:
    print('Skipping dumping/loading of Ratio')


print('\nDone\n')

## Perform / Load plateau fits on the Ratio

We perform constant fits on symmetrical ranges of the ratio for each value of $t_s$. In selecting the best plateau fit range for every $t_{s}$, the criterion is to select the first fit range of which $\chi^2<0.8$.

In [ ]:
plateauFit_Source = {'Evaluate': False, 'Dump': False, 'Load': True}

Mplat_bins = {}
Mplat_ave  = {}

chiPlat_bins = {}
chiPlat_ave  = {}

optPlatFit = {}


### Create generic structures

In [ ]:
if plateauFit_Source['Evaluate']:

    fitInfo_RP = {}
    Nfits_RP = {}
    for opT in SrcSnkOpTypes:
        for phT in phaseTypes:
            ophTpl = (phT,opT)
            fitInfo_RP[ophTpl] = {}
            Nfits_RP[ophTpl] = {}
            for tsnk in TsinkList[ophTpl]:
                fitInfo_RP[ophTpl][tsnk] = {}
                Nfits_RP[ophTpl][tsnk] = tsnk//2-1
                fitInfo_RP[ophTpl][tsnk]['Nfits'] = Nfits_RP[ophTpl][tsnk]

                tini = 1 # Leave out the source point
                tfin = tsnk-1
                fitInfo_RP[ophTpl][tsnk]['Rng'] = []
                for nf in range(Nfits_RP[ophTpl][tsnk]):
                    fitInfo_RP[ophTpl][tsnk]['nf=%d'%(nf)] = {}
                    tstart = tini+nf
                    tstop  = tfin-nf
                    Npts = tstop-tstart+1
                    fitInfo_RP[ophTpl][tsnk]['nf=%d'%(nf)]['xdata']  = np.arange(Npts+1)
                    fitInfo_RP[ophTpl][tsnk]['nf=%d'%(nf)]['tstart'] = tstart
                    fitInfo_RP[ophTpl][tsnk]['nf=%d'%(nf)]['tstop']  = tstop
                    fitInfo_RP[ophTpl][tsnk]['nf=%d'%(nf)]['Npts']   = Npts

                    fitInfo_RP[ophTpl][tsnk]['Rng'].append('%d-%d'%(tstart,tstop))

print('Done\n')

### Perform plateau fits

In [ ]:

if plateauFit_Source['Evaluate']:

    for ri in ReIm:
        Mplat_bins[ri] = {}
        Mplat_ave[ri]  = {}

        chiPlat_bins[ri] = {}
        chiPlat_ave[ri]  = {}

        optPlatFit[ri] = {}

    chiCrit = 0.8

    for opT in SrcSnkOpTypes:
        for phT in phaseTypes:
            ophTpl = (phT,opT)
            momTags = momTags_Dict_ave[ophTpl]

            for mTag in momTags:
                mVec = momVector_Dict[mTag]
                zMom = mVec[2]
                mFTag = momFileTag(mVec)
                pmTpl = phMomTpl(phT,zMom)

                srcOpList = srcOpList3pt_Dict[opT][pmTpl] # These are the same in two- and
                snkOpList = srcOpList3pt_Dict[opT][pmTpl] # three-point functions

                for tsnk in TsinkList[(phT,opT)]:                        
                    dTpl = (phT,opT,mTag,tsnk)
                    Nbins = Nbins_3pt_Dict[dTpl]

                    adTplM = (phT,opT,mTag,tsnk)
                    if analyzeDataSetM[adTplM]:

                        for isrc, src in enumerate(srcOpList):
                            for isnk, snk in enumerate(snkOpList):                        
                                # Only upolarized PDF for now
                                irowSrc = 1
                                irowSnk = 1
                                gMat = 'gt'

                                for z in dispLen_ave:
                                    ckey3 = (opT,phT,mTag,tsnk,isrc,isnk,gMat,z,irowSrc,irowSnk)

                                    for ri in ReIm:
                                        Mplat_bins[ri][ckey3] = {}
                                        Mplat_ave[ri][ckey3] = {}
                                        chiPlat_bins[ri][ckey3] = {}
                                        chiPlat_ave[ri][ckey3] = {}
                                        optPlatFitFound = False
                                        optPlatFit[ri][ckey3] = None
                                        for nf in range(fitInfo_RP[ophTpl][tsnk]['Nfits']):
                                            tstart = fitInfo_RP[ophTpl][tsnk]['nf=%d'%(nf)]['tstart']
                                            tstop  = fitInfo_RP[ophTpl][tsnk]['nf=%d'%(nf)]['tstop']
                                            Npts   = fitInfo_RP[ophTpl][tsnk]['nf=%d'%(nf)]['Npts']

                                            # Doesn't matter for const. fit
                                            xdata = fitInfo_RP[ophTpl][tsnk]['nf=%d'%(nf)]['xdata']

                                            Mplat_bins[ri][ckey3][nf] = np.zeros(Nbins)
                                            chiPlat_bins[ri][ckey3][nf] = np.zeros(Nbins)
                                            for nb in range(Nbins):
                                                # Set fit values and error
                                                fitVal = RA_bins[ri][ckey3][nb,tstart:tstop+1]
                                                fitErr = RA_ave[ri][ckey3][1][tstart:tstop+1]

                                                # Perform plateau fits
                                                Mplat_bins[ri][ckey3][nf][nb], plCov = scipyOpt.curve_fit(constFit,xdata,fitVal,sigma=fitErr)

                                                # Calculate chi^2
                                                chiPlat_bins[ri][ckey3][nf][nb] = chiSq_const(xdata, fitVal, fitErr, Mplat_bins[ri][ckey3][nf][nb])
                                                chiPlat_bins[ri][ckey3][nf][nb] /= (Npts-2)
                                            # for- Nbins

                                            Mplat_ave[ri][ckey3][nf]   = jackknife_ave(Mplat_bins[ri][ckey3][nf],Nbins,tL=1)
                                            chiPlat_ave[ri][ckey3][nf] = jackknife_ave(chiPlat_bins[ri][ckey3][nf],Nbins,tL=1)

                                            if chiPlat_ave[ri][ckey3][nf][0] <= chiCrit and not optPlatFitFound:
                                                optPlatFit[ri][ckey3] = nf
                                                optPlatFitFound = True
                                        # for Nfits                                    
                    # if analyze

                    print('Done: Momentum %s , tsnk %d'%(mTag,tsnk))
else:
    print('Skipping performing the plateau fits')
                    
print('\nDone\n')

In [ ]:
if plateauFit_Source['Dump']:
    print('Dumping the Plateau fits...')
    pickle.dump(Mplat_bins, open('Mplat_bins.p', 'wb' ))
    pickle.dump(Mplat_ave, open('Mplat_ave.p', 'wb' ))
    pickle.dump(chiPlat_bins, open('chiPlat_bins.p', 'wb' ))
    pickle.dump(chiPlat_ave, open('chiPlat_ave.p', 'wb' ))
    pickle.dump(optPlatFit, open('optPlatFit.p', 'wb' ))
elif plateauFit_Source['Load']:
    print('Loading the Plateau fits...')
    Mplat_bins = pickle.load(open('Mplat_bins.p', 'rb'))
    Mplat_ave = pickle.load(open('Mplat_ave.p', 'rb'))
    chiPlat_bins = pickle.load(open('chiPlat_bins.p', 'rb'))
    chiPlat_ave = pickle.load(open('chiPlat_ave.p', 'rb'))
    optPlatFit = pickle.load(open('optPlatFit.p', 'rb'))

    Nbins_3pt_Dict = pickle.load( open('Nbins_3pt_Dict.p', 'rb'))
else:
    print('Skipping dumping/loading of the Plateau fits')


print('\nDone\n')

In [ ]:
# Vector charge

opTR = 'single'
phTR = 'unphased'
isrcR = 0
isnkR = 0
irowSrcR = 1
irowSnkR = 1

gMatR = 'gt'
mTagR = 'pz.0'
dL = 0

tsink = 12
ckey = (opTR,phTR,mTagR,tsink,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)
ckey_ZV = (opTR,phTR,'pz.0',10,0,0,'gt',0,1,1)

ZV = Mplat_ave['Re'][ckey_ZV][0]

print('Zv = ', ZV)
    

## Perform/Load summation-method fits on the Ratio

Below we perform 4 types of fits on the summed ratio, $S(t_s) = \sum_{t_{ins}=1}^{t_s-1} \frac{C_{3pt}(t_s,t_{ins})}{C_{2pt}(t_s)}$

(i) Linear fit of the form: $S(t_s) = Z + M t_s$, where $M$ is the matrix element, with $t_s$ starting from $4a,6a,8a,10a$.

(ii) 1-exponential fit of the form: $S(t_s) = Z + M t_s + At_s e^{-\Delta Et_s}$, where $M$ is the matrix element

(iii) Constant fit on the "reduced" summed ratio, $S_r(t_s) = S(t_s+1) - S(t_s)$, keeping and ignoring the lowest $t_s$ point

(iv) 1-exponential fit on the reduced summed ratio, of the form: $S_r(t_s) = M + Ae^{-\Delta E t_s}$

In [ ]:
summFit_Source = {'Evaluate': False, 'Dump': False, 'Load': True}

# 1-Exponential fit on reduced-summed ratio
E1rsum_fP = ['M','A','dE']
E1rsum_bins = {}
E1rsum_ave = {}
E1rsum_chiAve = {}
nfrE1 = {}

# Constant fit on reduced-summed ratio keeping lowest (Ml) and ignoring lowest (Mh) point
Crsum_bins = {}
Crsum_ave  = {}
Crsum_chiAve = {}

# Linear fit on regular-summed ratio
Lsum_fP = ['M','b']
Lsum_fR = [4,6,8,10]
Lsum_bins = {}
Lsum_ave = {}
Lsum_chiAve = {}

# 1-Exponential fit on regular-summed ratio
E1sum_fP = ['M','Z','A','dE']
E1sum_bins = {}
E1sum_ave = {}
E1sum_chiAve = {}
nfE1 = {}


In [ ]:

if summFit_Source['Evaluate']:

    # 1-Exponential fit on reduced-summed ratio
    for fp in E1rsum_fP:
        E1rsum_bins[fp] = {'Re': {}, 'Im': {}}
        E1rsum_ave[fp]  = {'Re': {}, 'Im': {}}
    for ri in ReIm:
        E1rsum_chiAve[ri] = {}
        nfrE1[ri] = {}
    #------------------------

    # Constant fit on reduced-summed ratio keeping lowest (Ml) and ignoring lowest (Mh) point
    for rsf in ['l','h']:
        dps = 'M_'+rsf
        Crsum_bins[dps]   = {'Re': {}, 'Im': {}}
        Crsum_ave[dps]    = {'Re': {}, 'Im': {}}
        Crsum_chiAve[rsf] = {'Re': {}, 'Im': {}}
    #------------------------

    # Linear fit on regular-summed ratio
    for sf in Lsum_fR:
        sfs = 'l%d'%(sf)
        for fp in Lsum_fP:
            dps = fp+'_%s'%(sfs)
            Lsum_bins[dps]  = {'Re': {}, 'Im': {}}
            Lsum_ave[dps]   = {'Re': {}, 'Im': {}}
        Lsum_chiAve[sfs] = {'Re': {}, 'Im': {}}
    #------------------------

    # 1-Exponential fit on regular-summed ratio
    for fp in E1sum_fP:
        E1sum_bins[fp] = {'Re': {}, 'Im': {}}
        E1sum_ave[fp]  = {'Re': {}, 'Im': {}}
    for ri in ReIm:
        E1sum_chiAve[ri] = {}
        nfE1[ri] = {}


    for opT in SrcSnkOpTypes:
        for phT in phaseTypes:
            momTags = momTags_Dict_ave[(phT,opT)]

            # Fit ranges for reduced-summation fits
            TsinkList_rs = {'l': TsinkList[(phT,opT)][:-1],'h': TsinkList[(phT,opT)][1:-1]}
            Nx_rs = {}
            xData_rs = {}
            for rsf in ['l','h']:
                Nx_rs[rsf] = len(TsinkList_rs[rsf])
                xData_rs[rsf] = TsinkList_rs[rsf]


            # Fit ranges for linear fits on regular summed ratio fits
            TsinkList_sL = {}
            Nx_sL = {}
            xData_sL = {}
            for isf,sf in enumerate(Lsum_fR):
                sfs = 'l%d'%(sf)
                isf = TsinkList[(phT,opT)].index(sf)
                TsinkList_sL[sfs] = TsinkList[(phT,opT)][isf:]
                Nx_sL[sfs] = len(TsinkList_sL[sfs])
                xData_sL[sfs] = TsinkList_sL[sfs]


            # Fit range for the exponential fit on regular summed-ratio
            TsinkList_sE = TsinkList[(phT,opT)]
            Nx_sE = len(TsinkList_sE)
            xData_sE = TsinkList_sE

            for mTag in momTags:
                mVec = momVector_Dict[mTag]
                zMom = mVec[2]
                mFTag = momFileTag(mVec)
                pmTpl = phMomTpl(phT,zMom)

                srcOpList = srcOpList3pt_Dict[opT][pmTpl] # These are the same in two- and
                snkOpList = srcOpList3pt_Dict[opT][pmTpl] # three-point functions

                for isrc, src in enumerate(srcOpList):
                    for isnk, snk in enumerate(snkOpList):
                        dTpl = (phT,opT,mTag,tsnk)
                        Nbins = Nbins_3pt_Dict[dTpl]

                        irowSrc = 1
                        irowSnk = 1
                        gMat = 'gt'
                        for z in dispLen_ave:
                            ckeyF = (opT,phT,mTag,isrc,isnk,gMat,z,irowSrc,irowSnk)

                            for ri in ReIm:
                                # Constant fit on reduced-summed ratio
                                Crsum_chiBins = {}
                                for rsf in ['l','h']:
                                    dps = 'M_'+rsf
                                    Crsum_bins[dps][ri][ckeyF] = np.zeros(Nbins)
                                    Crsum_chiBins[rsf] = np.zeros(Nbins)

                                # 1-exponential fit on regular-summed ratio
                                for fp in E1sum_fP:
                                    E1sum_bins[fp][ri][ckeyF]  = np.zeros(Nbins)
                                E1sum_chiBins  = np.zeros(Nbins)

                                # 1-exponential fit on reduced-summed ratio
                                for fp in E1rsum_fP:
                                    E1rsum_bins[fp][ri][ckeyF] = np.zeros(Nbins)
                                E1rsum_chiBins = np.zeros(Nbins)

                                # linear fit on regular summed ratio
                                Lsum_chiBins = {}
                                for sf in Lsum_fR:
                                    sfs = 'l%d'%(sf)
                                    for fp in Lsum_fP:
                                        dps = fp+'_%s'%(sfs)
                                        Lsum_bins[dps][ri][ckeyF] = np.zeros(Nbins)
                                    Lsum_chiBins[sfs] = np.zeros(Nbins)

                                nfE1_c  = 0
                                nfrE1_c = 0
                                for nb in range(Nbins):

                                    #----------------------------------
                                    # Fits for the reduced-summed ratio
                                    #----------------------------------

                                    for rsf in ['h','l']:
                                        dps = 'M_'+rsf
                                        rsum_fitVal = np.zeros(Nx_rs[rsf])
                                        rsum_fitErr = np.zeros(Nx_rs[rsf])
                                        for its,tsnk in enumerate(TsinkList_rs[rsf]):
                                            ckey3 = (opT,phT,mTag,tsnk,isrc,isnk,gMat,z,irowSrc,irowSnk) # 3pt-tuple
                                            rsum_fitVal[its] = RrsumA_bins[ri][ckey3][nb]                                    
                                            rsum_fitErr[its] = RrsumA_ave[ri][ckey3][1]                                   


                                        # Perform constant fit and calculate chi^2
                                        Crsum_bins[dps][ri][ckeyF][nb], rsCCov_Re = scipyOpt.curve_fit(constFit,xData_rs[rsf],rsum_fitVal,sigma=rsum_fitErr)
                                        Crsum_chiBins[rsf][nb] = chiSq_const(xData_rs[rsf], rsum_fitVal, rsum_fitErr, Crsum_bins[dps][ri][ckeyF][nb])
                                        Crsum_chiBins[rsf][nb] /= (Nx_rs[rsf]-2)


                                        # Perform 1-exponential fit and calculate chi^2
                                        if rsf == 'l':

                                            # Take care of initial guess
                                            guess_rsE1 = np.zeros(3)
                                            guess_rsE1[0] = Crsum_bins['M_h'][ri][ckeyF][nb]

                                            # Take special cases into account
                                            if( (ri == 'Im') and 
                                                ((mTag == 'pz.+1') and (z==1 or z==2 or z==6 or z==7)) or
                                                ((mTag == 'pz.+2') and (z==1)) or
                                                ((mTag == 'pz.+3') and (z==1)) ):
                                                mTagG0 = MomTag(0)
                                                zG0 = z
                                                ckeyG0  = (opT,phT,mTagG0,isrc,isnk,gMat,zG0,irowSrc,irowSnk)
                                                for ifp,fp in enumerate(['A','dE']):
                                                    testG0  = E1rsum_ave[fp][ri][ckeyG0][0]
                                                    if testG0 != None:
                                                        guess_rsE1[ifp+1] = testG0
                                                    else:
                                                        raise ValueError('Test guess for %s,z%d is None'%(mTag,z))
                                            else:
                                                if z != 0:
                                                    zG = z-1
                                                    zG0 = 0
                                                    if mTag != 'pz.0':
                                                        mTagG0 = MomTag(0)
                                                        ckeyG   = (opT,phT,mTag  ,isrc,isnk,gMat,zG ,irowSrc,irowSnk)
                                                        ckeyG0  = (opT,phT,mTagG0,isrc,isnk,gMat,zG ,irowSrc,irowSnk)
                                                        ckeyG00 = (opT,phT,mTagG0,isrc,isnk,gMat,zG0,irowSrc,irowSnk)
                                                        for ifp,fp in enumerate(['A','dE']):
                                                            testG   = E1rsum_ave[fp][ri][ckeyG][0]
                                                            testG0  = E1rsum_ave[fp][ri][ckeyG0][0]
                                                            testG00 = E1rsum_ave[fp][ri][ckeyG00][0]
                                                            if testG0 != None:
                                                                guess_rsE1[ifp+1] = testG0
                                                            else:
                                                                if testG00 != None:
                                                                    guess_rsE1[ifp+1] = testG00
                                                                else:
                                                                    guess_rsE1[ifp+1] = testG
                                                    else:
                                                        ckeyG   = (opT,phT,mTag,isrc,isnk,gMat,zG ,irowSrc,irowSnk)
                                                        ckeyG0  = (opT,phT,mTag,isrc,isnk,gMat,zG0,irowSrc,irowSnk)
                                                        for ifp,fp in enumerate(['A','dE']):
                                                            testG   = E1rsum_ave[fp][ri][ckeyG][0]
                                                            testG0  = E1rsum_ave[fp][ri][ckeyG0][0]
                                                            if testG != None:
                                                                guess_rsE1[ifp+1] = testG
                                                            else:
                                                                guess_rsE1[ifp+1] = testG0
                                                # if z != 0
                                                else:
                                                    if mTag != 'pz.0':
                                                        mTagG = MomTag(0)
                                                        ckeyG = (opT,phT,mTagG,isrc,isnk,gMat,z,irowSrc,irowSnk)
                                                        for ifp,fp in enumerate(['A','dE']):
                                                            testG = E1rsum_ave[fp][ri][ckeyG][0]
                                                            if testG != None:
                                                                guess_rsE1[ifp+1] = testG
                                                            else:
                                                                guess_rsE1[ifp+1] = 0.1
                                                    else:
                                                        guess_rsE1[1] = 0.1 # A
                                                        guess_rsE1[2] = 0.1 # dE
                                            #-----------------------------

                                            try:
                                                # Perform fit
                                                fitParamsE1r, sE1rCov = scipyOpt.curve_fit(exp1rFit,xData_rs[rsf],rsum_fitVal,sigma=rsum_fitErr,p0=guess_rsE1)
                                                for ifp,fp in enumerate(E1rsum_fP):
                                                    E1rsum_bins[fp][ri][ckeyF][nfrE1_c] = fitParamsE1r[ifp]
                                                E1rsum_chiBins[nfrE1_c] = chiSq_exp1r(xData_rs[rsf], rsum_fitVal, rsum_fitErr, E1rsum_bins['M'][ri][ckeyF][nb],E1rsum_bins['A'][ri][ckeyF][nfE1_c],E1rsum_bins['dE'][ri][ckeyF][nfE1_c])
                                                E1rsum_chiBins[nfrE1_c] /= (Nx_rs[rsf]-4)
                                                nfrE1_c += 1
                                            except:
                                                pass

                                    # for low-high
                                    #---------------------------------------------


                                    #----------------------------------
                                    # Fits for the regular-summed ratio
                                    #----------------------------------

                                    # Perform linear fit and calculate chi^2
                                    for isf,sf in enumerate(Lsum_fR):
                                        sfs = 'l%d'%(sf)

                                        sum_fitVal = np.zeros(Nx_sL[sfs])
                                        sum_fitErr = np.zeros(Nx_sL[sfs])
                                        for its,tsnk in enumerate(TsinkList_sL[sfs]):
                                            ckey3 = (opT,phT,mTag,tsnk,isrc,isnk,gMat,z,irowSrc,irowSnk) # 3pt-tuple
                                            sum_fitVal[its] = RsumA_bins[ri][ckey3][nb]                                    
                                            sum_fitErr[its] = RsumA_ave[ri][ckey3][1]                                   


                                        fitParamsL, rsLCov = scipyOpt.curve_fit(linearFit,xData_sL[sfs],sum_fitVal,sigma=sum_fitErr)
                                        for ifp,fp in enumerate(Lsum_fP):
                                            dps = fp+'_%s'%(sfs)
                                            Lsum_bins[dps][ri][ckeyF][nb] = fitParamsL[ifp]
                                        Lsum_chiBins[sfs][nb] = chiSq_linear(xData_sL[sfs], sum_fitVal, sum_fitErr, Lsum_bins['M'+'_%s'%(sfs)][ri][ckeyF][nb],Lsum_bins['b'+'_%s'%(sfs)][ri][ckeyF][nb])
                                        Lsum_chiBins[sfs][nb] /= (Nx_sE-3)
                                    #-------------------


                                    # Perform 1-exponential fit and calculate chi^2
                                    sum_fitVal = np.zeros(Nx_sE)
                                    sum_fitErr = np.zeros(Nx_sE)
                                    for its,tsnk in enumerate(TsinkList_sE):
                                        ckey3 = (opT,phT,mTag,tsnk,isrc,isnk,gMat,z,irowSrc,irowSnk) # 3pt-tuple
                                        sum_fitVal[its] = RsumA_bins[ri][ckey3][nb]                                    
                                        sum_fitErr[its] = RsumA_ave[ri][ckey3][1]                                   

                                    guess_sE1 = np.zeros(4)
                                    guess_sE1[0] = Lsum_bins['M_l8'][ri][ckeyF][nb]                                                
                                    if nfE1_c == 0:
                                        if z != 0:
                                            ckeyG = (opT,phT,mTag,isrc,isnk,gMat,z-1,irowSrc,irowSnk)
                                            for ifp,fp in enumerate(['Z','A','dE']):
                                                testG = E1sum_ave[fp][ri][ckeyG][0]
                                                if testG != None:
                                                    guess_sE1[ifp+1] = testG
                                                else:
                                                    guess_sE1[ifp+1] = 0.1
                                        else:
                                            if mTag != 'pz.0':
                                                mTagG = MomTag(mVec[2]-1)
                                                ckeyG = (opT,phT,mTagG,isrc,isnk,gMat,z,irowSrc,irowSnk)
                                                for ifp,fp in enumerate(['Z','A','dE']):
                                                    testG = E1sum_ave[fp][ri][ckeyG][0]
                                                    if testG != None:
                                                        guess_sE1[ifp+1] = testG
                                                    else:
                                                        guess_sE1[ifp+1] = 0.1
                                            else:
                                                for ifp,fp in enumerate(['Z','A','dE']):
                                                    guess_sE1[ifp+1] = 0.1
                                    else:
                                        for ifp,fp in enumerate(['Z','A','dE']):
                                            guess_sE1[ifp+1] = E1sum_bins[fp][ri][ckeyF][nfE1_c-1]
                                    #-----------------------------


                                    try:
                                        fitParamsE1, sE1Cov = scipyOpt.curve_fit(exp1Fit,xData_sE,sum_fitVal,sigma=sum_fitErr,p0=guess_sE1)
                                        for ifp,fp in enumerate(E1sum_fP):
                                            E1sum_bins[fp][ri][ckeyF][nfE1_c] = fitParamsE1[ifp]
                                        E1sum_chiBins[nfE1_c] = chiSq_exp1(xData_sE, sum_fitVal, sum_fitErr, E1sum_bins['M'][ri][ckeyF][nb],E1sum_bins['Z'][ri][ckeyF][nfE1_c],E1sum_bins['A'][ri][ckeyF][nfE1_c],E1sum_bins['dE'][ri][ckeyF][nfE1_c])
                                        E1sum_chiBins[nfE1_c] /= (Nx_sE-5)
                                        nfE1_c += 1
                                    except:
                                        pass
                                # for-Nbins ---------------------------------------


                                #-------------
                                # Fit averages
                                #-------------

                                # Reduced summed-ratio Constant fit averages
                                for rsf in ['l','h']:
                                    dps = 'M_'+rsf
                                    Crsum_ave[dps][ri][ckeyF] = jackknife_ave(Crsum_bins[dps][ri][ckeyF],Nbins,tL=1)
                                    Crsum_chiAve[rsf][ri][ckeyF] = jackknife_ave(Crsum_chiBins[rsf],Nbins,tL=1)
                                #----------------------

                                # Summed-ratio Linear fit averages
                                for isf,sf in enumerate(Lsum_fR):
                                    sfs = 'l%d'%(sf)

                                    for fp in Lsum_fP:
                                        dps = fp+'_%s'%(sfs)                                    
                                        Lsum_ave[dps][ri][ckeyF] = jackknife_ave(Lsum_bins[dps][ri][ckeyF],Nbins,tL=1)
                                    Lsum_chiAve[sfs][ri][ckeyF] = jackknife_ave(Lsum_chiBins[sfs],Nbins,tL=1)
                                #----------------------


                                # 1-exponential fit averages
                                nfE1[ri][ckeyF]  = nfE1_c
                                nfrE1[ri][ckeyF] = nfrE1_c

                                for fp in E1sum_fP:
                                    E1sum_bins[fp][ri][ckeyF]  = E1sum_bins[fp][ri][ckeyF][:nfE1_c]
                                for fp in E1rsum_fP:
                                    E1rsum_bins[fp][ri][ckeyF] = E1rsum_bins[fp][ri][ckeyF][:nfrE1_c]

                                E1sum_chiBins  = E1sum_chiBins[:nfE1_c]
                                E1rsum_chiBins = E1rsum_chiBins[:nfrE1_c]


                                if nfE1_c == 0: # No successful fits
                                    for fp in E1sum_fP:
                                         E1sum_ave[fp][ri][ckeyF] = (None,None)
                                    E1sum_chiAve[ri][ckeyF] = (None,None)
                                elif nfE1_c == 1: # One successful fit
                                    for fp in E1sum_fP:
                                         E1sum_ave[fp][ri][ckeyF] = (E1sum_bins[fp][ri][ckeyF][0],0.0)
                                    E1sum_chiAve[ri][ckeyF] = (E1sum_chiBins[0], 0.0)
                                else: # More than one successful fits
                                    for fp in E1sum_fP:
                                        E1sum_ave[fp][ri][ckeyF] = jackknife_ave(E1sum_bins[fp][ri][ckeyF],nfE1_c,tL=1)
                                    E1sum_chiAve[ri][ckeyF] = jackknife_ave(E1sum_chiBins,nfE1_c,tL=1)
                                #----------------------

                                if nfrE1_c == 0: # No successful fits
                                    for fp in E1rsum_fP:
                                         E1rsum_ave[fp][ri][ckeyF] = (None,None)
                                    E1rsum_chiAve[ri][ckeyF] = (None,None)
                                elif nfrE1_c == 1: # One successful fit
                                    for fp in E1rsum_fP:
                                         E1rsum_ave[fp][ri][ckeyF] = (E1rsum_bins[fp][ri][ckeyF][0],0.0)
                                    E1rsum_chiAve[ri][ckeyF] = (E1rsum_chiBins[0], 0.0)
                                else: # More than one successful fits
                                    for fp in E1rsum_fP:
                                        E1rsum_ave[fp][ri][ckeyF] = jackknife_ave(E1rsum_bins[fp][ri][ckeyF],nfrE1_c,tL=1)
                                    E1rsum_chiAve[ri][ckeyF] = jackknife_ave(E1rsum_chiBins,nfrE1_c,tL=1)
                                #----------------------

                                if not (ri=='Im' and z*mVec[2]==0):
                                    print('%s- S: z = %d: nfE1_c  = %d: '%(ri,z,nfE1_c), E1sum_ave['M'][ri][ckeyF] )
                                    print('%s-RS: z = %d: nfrE1_c = %d: '%(ri,z,nfrE1_c),E1rsum_ave['M'][ri][ckeyF] )


                print('Done: Momentum %s'%(mTag))
else:
    print('Skipping performing the summation method fits')
                
print('\nDone\n')

In [ ]:
if summFit_Source['Dump']:
    print('Dumping the summation method fits...')
    pickle.dump(E1rsum_bins, open('E1rsum_bins.p', 'wb' ))
    pickle.dump(E1rsum_ave, open('E1rsum_ave.p', 'wb' ))
    pickle.dump(E1rsum_chiAve, open('E1rsum_chiAve.p', 'wb' ))
    pickle.dump(nfrE1, open('nfrE1.p', 'wb' ))

    pickle.dump(Crsum_bins, open('Crsum_bins.p', 'wb' ))
    pickle.dump(Crsum_ave, open('Crsum_ave.p', 'wb' ))
    pickle.dump(Crsum_chiAve, open('Crsum_chiAve.p', 'wb' ))

    pickle.dump(Lsum_bins, open('Lsum_bins.p', 'wb' ))
    pickle.dump(Lsum_ave, open('Lsum_ave.p', 'wb' ))
    pickle.dump(Lsum_chiAve, open('Lsum_chiAve.p', 'wb' ))

    pickle.dump(E1sum_bins, open('E1sum_bins.p', 'wb' ))
    pickle.dump(E1sum_ave, open('E1sum_ave.p', 'wb' ))
    pickle.dump(E1sum_chiAve, open('E1sum_chiAve.p', 'wb' ))
    pickle.dump(nfE1, open('nfE1.p', 'wb' ))
    
elif summFit_Source['Load']:
    print('Loading the summation method fits...')

    E1rsum_bins = pickle.load(open('E1rsum_bins.p', 'rb'))
    E1rsum_ave = pickle.load(open('E1rsum_ave.p', 'rb'))
    E1rsum_chiAve = pickle.load(open('E1rsum_chiAve.p', 'rb'))
    nfrE1 = pickle.load(open('nfrE1.p', 'rb'))

    Crsum_bins = pickle.load(open('Crsum_bins.p', 'rb'))
    Crsum_ave = pickle.load(open('Crsum_ave.p', 'rb'))
    Crsum_chiAve = pickle.load(open('Crsum_chiAve.p', 'rb'))

    Lsum_bins = pickle.load(open('Lsum_bins.p', 'rb'))
    Lsum_ave = pickle.load(open('Lsum_ave.p', 'rb'))
    Lsum_chiAve = pickle.load(open('Lsum_chiAve.p', 'rb'))

    E1sum_bins = pickle.load(open('E1sum_bins.p', 'rb'))
    E1sum_ave = pickle.load(open('E1sum_ave.p', 'rb'))
    E1sum_chiAve = pickle.load(open('E1sum_chiAve.p', 'rb'))
    nfE1 = pickle.load(open('nfE1.p', 'rb'))

    Nbins_3pt_Dict = pickle.load( open('Nbins_3pt_Dict.p', 'rb'))
else:
    print('Skipping dumping/loading of the summation method fits')


print('\nDone\n')


### Create error bands for summation method fits

In [ ]:
summBands_Source = {'Evaluate': False, 'Dump': False, 'Load': True}

sumL_band = {}
    
sumE1_band  = {}
rsumE1_band = {}


In [ ]:

if summBands_Source['Evaluate']:

    Nbpts = 251

    for sf in Lsum_fR:
        sfs = 'l%d'%(sf)
        sumL_band[sfs] = {'Re':{}, 'Im':{}}


    for ri in ReIm:
        sumE1_band[ri]  = {}
        rsumE1_band[ri] = {}


    for opT in SrcSnkOpTypes:
        for phT in phaseTypes:
            momTags = momTags_Dict_ave[(phT,opT)]

            # Fit ranges for reduced-summation fits
            TsinkList_rs = {'l': TsinkList[(phT,opT)][:-1],'h': TsinkList[(phT,opT)][1:-1]}
            
            # Fit ranges for linear fits on regular summed ratio fits
            TsinkList_sL = {}
            for isf,sf in enumerate(Lsum_fR):
                sfs = 'l%d'%(sf)
                isf = TsinkList[(phT,opT)].index(sf)
                TsinkList_sL[sfs] = TsinkList[(phT,opT)][isf:]

            # Fit range for the exponential fit on regular summed-ratio
            TsinkList_sE = TsinkList[(phT,opT)]
            
            
            # Summed-ratio linear fit ranges
            xStart_L = {}
            dx_L = {}
            for sf in Lsum_fR:
                sfs = 'l%d'%(sf)
                xStart_L[sfs] = TsinkList_sL[sfs][0]
                xEnd_L        = TsinkList_sL[sfs][-1]+1
                dx_L[sfs]     = (xEnd_L - xStart_L[sfs])  / (Nbpts-1)

            # Summed-ratio exp-fit ranges
            xStart_E1 = TsinkList[(phT,opT)][0]-1
            xEnd_E1   = TsinkList[(phT,opT)][-1]+1
            dx_E1     = (xEnd_E1 - xStart_E1)  / (Nbpts-1)


            # Reduced-summed ratio exp-fit ranges
            xStart_E1r = TsinkList_rs['l'][0]-1
            xEnd_E1r   = TsinkList_rs['l'][-1]+1
            dx_E1r = (xEnd_E1r - xStart_E1r) / (Nbpts-1)

            for mTag in momTags:
                mVec = momVector_Dict[mTag]
                zMom = mVec[2]
                mFTag = momFileTag(mVec)
                pmTpl = phMomTpl(phT,zMom)

                srcOpList = srcOpList3pt_Dict[opT][pmTpl] # These are the same in two- and
                snkOpList = srcOpList3pt_Dict[opT][pmTpl] # three-point functions

                for isrc, src in enumerate(srcOpList):
                    for isnk, snk in enumerate(snkOpList):
                        dTpl = (phT,opT,mTag,tsnk)
                        Nbins = Nbins_3pt_Dict[dTpl]

                        irowSrc = 1
                        irowSnk = 1
                        gMat = 'gt'
                        for z in dispLen_ave:
                            ckeyF = (opT,phT,mTag,isrc,isnk,gMat,z,irowSrc,irowSnk)

                            for ri in ReIm:
                                for sf in Lsum_fR:
                                    sfs = 'l%d'%(sf)
                                    sumL_band[sfs][ri][ckeyF]   = {'x'  : np.zeros(Nbpts),'v': np.zeros(Nbpts),'e': np.zeros(Nbpts)}

                                sumE1_band[ri][ckeyF]  = {'x'  : np.zeros(Nbpts),'v': np.zeros(Nbpts),'e': np.zeros(Nbpts)}
                                rsumE1_band[ri][ckeyF] = {'x'  : np.zeros(Nbpts),'v': np.zeros(Nbpts),'e': np.zeros(Nbpts)}
                                for ibp in range(Nbpts):

                                    # Linear fit
                                    for sf in Lsum_fR:
                                        sfs = 'l%d'%(sf)

                                        xi_L  = xStart_L[sfs]  + ibp*dx_L[sfs]

                                        dps = 'M_%s'%(sfs)
                                        if len(Lsum_bins[dps][ri][ckeyF]) != Nbins:
                                            raise ValueError('Incorrect size of bins for linear fit %s, ckey '%(sfs),ckeyF)


                                        sumL_band[sfs][ri][ckeyF]['x'][ibp] = xi_L
                                        sumL_band[sfs][ri][ckeyF]['v'][ibp] = linearFit(xi_L, Lsum_ave['M_%s'%(sfs)][ri][ckeyF][0], Lsum_ave['b_%s'%(sfs)][ri][ckeyF][0])

                                        bandVarL  = np.zeros(Nbins)
                                        for nb in range(Nbins):
                                            bandVarL[nb]  = linearFit(xi_L, Lsum_bins['M_%s'%(sfs)][ri][ckeyF][nb], Lsum_bins['b_%s'%(sfs)][ri][ckeyF][nb])
                                        sumL_band[sfs][ri][ckeyF]['e'][ibp]  = jackknife_ave(bandVarL ,Nbins,tL=1)[1]
                                    # for linear fit l4,l6,....


                                    # Exp-1 fit on regular summed ratio
                                    xi_E1  = xStart_E1 + ibp*dx_E1

                                    nfE1_c = nfE1[ri][ckeyF]

                                    sumE1_band[ri][ckeyF]['x'][ibp] = xi_E1
                                    if nfE1_c == 0:
                                        sumE1_band[ri][ckeyF]['v'][ibp] = None
                                        sumE1_band[ri][ckeyF]['e'][ibp] = None
                                    else:
                                        sumE1_band[ri][ckeyF]['v'][ibp] = exp1Fit(xi_E1, E1sum_ave['M'][ri][ckeyF][0], E1sum_ave['Z'][ri][ckeyF][0],E1sum_ave['A'][ri][ckeyF][0],E1sum_ave['dE'][ri][ckeyF][0])
                                        if nfE1_c == 1:
                                            sumE1_band[ri][ckeyF]['e'][ibp] = 0.0
                                        else:
                                            bandVarE1 = np.zeros(nfE1_c)
                                            for nb in range(nfE1_c):                                    
                                                bandVarE1[nb] = exp1Fit(xi_E1, E1sum_ave['M'][ri][ckeyF][0], E1sum_bins['Z'][ri][ckeyF][nb],E1sum_bins['A'][ri][ckeyF][nb],E1sum_bins['dE'][ri][ckeyF][nb])
                                            sumE1_band[ri][ckeyF]['e'][ibp] = jackknife_ave(bandVarE1,nfE1_c,tL=1)[1]


                                    # Exp-1 fit on reduced-summed ratio
                                    xi_E1r = xStart_E1r + ibp*dx_E1r
                                    nfrE1_c = nfrE1[ri][ckeyF]

                                    rsumE1_band[ri][ckeyF]['x'][ibp] = xi_E1r
                                    if nfrE1_c == 0:
                                        rsumE1_band[ri][ckeyF]['v'][ibp] = None
                                        rsumE1_band[ri][ckeyF]['e'][ibp] = None
                                    else:
                                        rsumE1_band[ri][ckeyF]['v'][ibp] = exp1rFit(xi_E1r, E1rsum_ave['M'][ri][ckeyF][0],E1rsum_ave['A'][ri][ckeyF][0],E1rsum_ave['dE'][ri][ckeyF][0])
                                        if nfE1_c == 1:
                                            rsumE1_band[ri][ckeyF]['e'][ibp] = 0.0
                                        else:
                                            bandVarE1r = np.zeros(nfrE1_c)
                                            for nb in range(nfrE1_c):                                    
                                                bandVarE1r[nb] = exp1rFit(xi_E1r, E1rsum_ave['M'][ri][ckeyF][0],E1rsum_bins['A'][ri][ckeyF][nb],E1rsum_bins['dE'][ri][ckeyF][nb])
                                            rsumE1_band[ri][ckeyF]['e'][ibp] = jackknife_ave(bandVarE1r,nfrE1_c,tL=1)[1]


                print('Done: Momentum %s'%(mTag))
else:
    print('Skipping evaluating the summation method fit bands')
                

print('\nDone\n')


In [ ]:
if summBands_Source['Dump']:
    print('Dumping the summation method fit bands...')
    pickle.dump(sumL_band, open('sumL_band.p', 'wb' ))
    pickle.dump(sumE1_band, open('sumE1_band.p', 'wb' ))
    pickle.dump(rsumE1_band, open('rsumE1_band.p', 'wb' ))    
elif summBands_Source['Load']:
    print('Loading the summation method fit bands...')
    sumL_band   = pickle.load(open('sumL_band.p', 'rb'))
    sumE1_band  = pickle.load(open('sumE1_band.p', 'rb'))
    rsumE1_band = pickle.load(open('rsumE1_band.p', 'rb'))
else:
    print('Skipping dumping/loading of the summation method fit bands')


print('\nDone\n')


### Plot summed ratio $R(t_s)$

In [ ]:
figRS, axRS = plt.subplots(2, 2, sharex='col', figsize=[15,9]) #, sharey='row'
figRS.subplots_adjust(hspace=0.00,wspace=0.2)#,right=0.8)


# Examples of fits:
# Pz = 2, z = 1
# Pz = 2, z = 2
# Pz = 2, z = 3
# Pz = 1, z = 1
# Pz = 1, z = 2
# Pz = 1, z = 3
# Pz = 1, z = 5
# Pz = 1, z = 7



opTRS = 'single'
phTRS = 'unphased'
isrcRS = 0
isnkRS = 0
irowSrcRS = 1
irowSnkRS = 1

gMatRS = 'gt'

mTagRS = 'pz.+1'
mVecRS = momVector_Dict[mTagRS]

dL = 2

# Plot summed-ratio
Ntsink = len(TsinkList[(phTRS,opTRS)])
for iri,ri in enumerate(ReIm):
    tTsinkAnalyze = []    
    RsumA_plot = (np.zeros(Ntsink),np.zeros(Ntsink))
    for its, tsink in enumerate(TsinkList[(phTRS,opTRS)]):
        ckey3 = (opTRS,phTRS,mTagRS,tsink,isrcRS,isnkRS,gMatRS,dL,irowSrcRS,irowSnkRS)

        adTplM = (phTRS,opTRS,mTagRS,tsink)
        if analyzeDataSetM[adTplM]:
            tTsinkAnalyze.append(tsink)
            for ive in range(2):
                RsumA_plot[ive][its] = RsumA_ave[ri][ckey3][ive]
    #----

    xRng = np.array(tTsinkAnalyze)

    axRS[iri][0].errorbar(xRng+osDctF[0],  RsumA_plot[0], yerr = RsumA_plot[1],
                 color = cDct[0], marker = mDct[0], ls = 'none',
                 ms = 8.0, elinewidth=elw, capsize=ecs,fillstyle='full')
    
    
    # Plot summed-ratio linear fit band
    ckeyF = (opTRS,phTRS,mTagRS,isrcRS,isnkRS,gMatRS,dL,irowSrcRS,irowSnkRS)

    for isf,sf in enumerate(Lsum_fR):
        sfs = 'l%s'%(sf)
        if ri == 'Re':
            lfLbl = r'linear fit $t_s^{\rm low} = %d$'%(sf)
        else:
            lfLbl = 'none'
        axRS[iri][0].plot(sumL_band[sfs][ri][ckeyF]['x'], sumL_band[sfs][ri][ckeyF]['v'], ls='-',color=cDct[isf],linewidth=0.8)
        axRS[iri][0].fill_between(sumL_band[sfs][ri][ckeyF]['x'], sumL_band[sfs][ri][ckeyF]['v']-sumL_band[sfs][ri][ckeyF]['e'], sumL_band[sfs][ri][ckeyF]['v']+sumL_band[sfs][ri][ckeyF]['e'], facecolor=cDct[isf], alpha=0.20,label=lfLbl)
    #-----------------------------
    
    
    # Plot summed-ratio 1-exponential fit band
    ckeyF = (opTRS,phTRS,mTagRS,isrcRS,isnkRS,gMatRS,dL,irowSrcRS,irowSnkRS)

    if ri == 'Re':
        e1fLbl = 'summ. 1-exp fit'
    else:
        e1fLbl = 'none'
    if not np.isnan(sumE1_band[ri][ckeyF]['v']).any():
        axRS[iri][0].plot(sumE1_band[ri][ckeyF]['x'], sumE1_band[ri][ckeyF]['v'], ls='-', color='slategray',linewidth=0.8)
        axRS[iri][0].fill_between(sumE1_band[ri][ckeyF]['x'], sumE1_band[ri][ckeyF]['v']-sumE1_band[ri][ckeyF]['e'], sumE1_band[ri][ckeyF]['v']+sumE1_band[ri][ckeyF]['e'], facecolor='slategray', alpha=0.20,label=e1fLbl)
    else:
        print('summ. 1-exp: Skipping Part = %s , Mom = %s , z3 = %d'%(ri,mTagRS,dL))
        
#----------------------------------------------------------------


# Plot reduced summed-ratio
Ntsink = len(TsinkList[(phTRS,opTRS)])

TsinkList_rs = {'l': TsinkList[(phTRS,opTRS)][:-1],'h': TsinkList[(phTRS,opTRS)][1:-1]}
TsinkList_plot = TsinkList[(phTRS,opTRS)][:-1]

for iri,ri in enumerate(ReIm):
    tTsinkAnalyze = []
    RrsumA_plot = (np.zeros(Ntsink-1),np.zeros(Ntsink-1))
    for its, tsink in enumerate(TsinkList_plot):
        ckey3 = (opTRS,phTRS,mTagRS,tsink,isrcRS,isnkRS,gMatRS,dL,irowSrcRS,irowSnkRS)

        adTplM = (phTRS,opTRS,mTagRS,tsink)
        if analyzeDataSetM[adTplM]:
            tTsinkAnalyze.append(tsink)
            for ive in range(2):
                RrsumA_plot[ive][its] = RrsumA_ave[ri][ckey3][ive]
    #--------------------------

    xRng = np.array(tTsinkAnalyze)

    axRS[iri][1].errorbar(xRng+osDct[0],  RrsumA_plot[0], yerr = RrsumA_plot[1],
                 color = cDct[0], marker = mDct[0], ls = 'none',
                 ms = 8.0, elinewidth=elw, capsize=ecs,fillstyle='full')
    
    
    # Plot regular-ratio linear fit slope
    ckeyF = (opTRS,phTRS,mTagRS,isrcRS,isnkRS,gMatRS,dL,irowSrcRS,irowSnkRS)

    for isf,sf in enumerate(Lsum_fR):
        sfs = 'l%s'%(sf)

        sumLVal = Lsum_ave['M_%s'%(sfs)][ri][ckeyF]

        xLo = 2+0.3*isf
        xHi = 14-0.3*isf
        xSt = (xHi-xLo)/10
        sumLX = np.arange(xLo,14+0.01,xSt)
        sumLY = np.array([sumLVal[0] for i in range(len(sumLX))])
        sumLYerr = np.array([sumLVal[1] for i in range(len(sumLX))])

        print('sum-L value-%s %s = '%(sfs,ri), sumLVal)

        if ri == 'Re':
            lfLbl = r'summ. linear fit slope $t_s^{\rm low} = %d$'%(sf)
        else:
            lfLbl = 'none'
        axRS[iri][1].plot(sumLX, sumLY, ls='-',color=cDct[isf],linewidth=1.0)
        axRS[iri][1].fill_between( sumLX , sumLY-sumLYerr, sumLY+sumLYerr, facecolor=cDct[isf], alpha=0.20)#,label=lfLbl)  
    #-----------------------------

    
    # Plot regular-ratio 1-exp fit matrix element
    ckeyF = (opTRS,phTRS,mTagRS,isrcRS,isnkRS,gMatRS,dL,irowSrcRS,irowSnkRS)

    sumE1Val = E1sum_ave['M'][ri][ckeyF]
    if sumE1Val[0] != None:
        if not np.isnan(sumE1Val[0]):

            sumE1X = np.arange(2,15)
            sumE1Y = np.array([sumE1Val[0] for i in range(len(sumE1X))])
            sumE1Yerr = np.array([sumE1Val[1] for i in range(len(sumE1X))])

            print('sum-E1 value %s = '%(ri), sumE1Val)

            if ri == 'Re':
                e1fLbl = 'summ. 1-exp fit'
            else:
                e1fLbl = 'none'
            axRS[iri][1].plot(sumE1X, sumE1Y, ls='-', color='slategray',linewidth=1.0)
            axRS[iri][1].fill_between( sumE1X , sumE1Y-sumE1Yerr, sumE1Y+sumE1Yerr, facecolor='slategray', alpha=0.25)#,label=e1fLbl)
    #-----------------------------

    
    # Plot reduced-summed ratio 1-exp fit matrix element
    ckeyF = (opTRS,phTRS,mTagRS,isrcRS,isnkRS,gMatRS,dL,irowSrcRS,irowSnkRS)

    rsumE1Val = E1rsum_ave['M'][ri][ckeyF]
    if rsumE1Val[0] != None:

        rsumE1X = xRng
        rsumE1Y = np.array([rsumE1Val[0] for i in range(len(rsumE1X))])
        rsumE1Yerr = np.array([rsumE1Val[1] for i in range(len(rsumE1X))])

        if ri == 'Re':
            re1fLbl = 'r-summ. 1-exp fit M'
        else:
            re1fLbl = 'none'
        axRS[iri][1].plot(rsumE1X, rsumE1Y, color='orange', ls='-',linewidth=1.0)
        axRS[iri][1].fill_between( rsumE1X , rsumE1Y-rsumE1Yerr, rsumE1Y+rsumE1Yerr, facecolor='orange', alpha=0.20,label=re1fLbl)
#     else:
#         print('Skipping Part = %s , Mom = %s , z3 = %d'%(ri,mTagRS,dL))        
    #-----------------------------
    
    
    # Plot reduced-summed ratio 1-exponential fit band
    ckeyF = (opTRS,phTRS,mTagRS,isrcRS,isnkRS,gMatRS,dL,irowSrcRS,irowSnkRS)
    
    if ri == 'Re':
        re1fLbl = 'r-summ. 1-exp fit'
    else:
        re1fLbl = 'none'
        
    if not np.isnan(rsumE1_band[ri][ckeyF]['v']).any():
        axRS[iri][1].plot(rsumE1_band[ri][ckeyF]['x'], rsumE1_band[ri][ckeyF]['v'], color='saddlebrown', ls='-',linewidth=0.8)
        axRS[iri][1].fill_between(rsumE1_band[ri][ckeyF]['x'], rsumE1_band[ri][ckeyF]['v']-rsumE1_band[ri][ckeyF]['e'], rsumE1_band[ri][ckeyF]['v']+rsumE1_band[ri][ckeyF]['e'], facecolor='saddlebrown', alpha=0.12,label=re1fLbl)
        rsumE1Val = E1rsum_ave['M'][ri][ckeyF]
        print('rsum-E1 value %s = '%(ri), rsumE1Val)
    else:
        print('Skipping Part = %s , Mom = %s , z3 = %d'%(ri,mTagRS,dL))
    #-----------------------------
    
    
    # Plot reduced-summed ratio constant fit keeping lowest point
    rsf = 'l'
    dps = 'M_'+rsf
    ckeyF = (opTRS,phTRS,mTagRS,isrcRS,isnkRS,gMatRS,dL,irowSrcRS,irowSnkRS)

    rsumCVal = Crsum_ave[dps][ri][ckeyF]

    rsumCX = TsinkList_rs[rsf]
    rsumCY = np.array([rsumCVal[0] for i in range(len(rsumCX))])
    rsumCYerr = np.array([rsumCVal[1] for i in range(len(rsumCX))])

    print('rsum-Cl value %s = '%(ri), rsumCVal)

    if ri == 'Re':
        cfLbl = 'r-summ. const. fit (L)'
    else:
        cfLbl = 'none'
    axRS[iri][1].plot(rsumCX, rsumCY, color='cyan', ls = '-',linewidth=1.0)
    axRS[iri][1].fill_between( rsumCX , rsumCY-rsumCYerr, rsumCY+rsumCYerr, facecolor='cyan', alpha=0.20,label=cfLbl)  
    #-----------------------------
    

    # Plot reduced-summed ratio constant fit ignoring lowest point
    rsf = 'h'
    dps = 'M_'+rsf
    ckeyF = (opTRS,phTRS,mTagRS,isrcRS,isnkRS,gMatRS,dL,irowSrcRS,irowSnkRS)

    rsumCVal = Crsum_ave[dps][ri][ckeyF]

    rsumCX = TsinkList_rs[rsf]
    rsumCY = np.array([rsumCVal[0] for i in range(len(rsumCX))])
    rsumCYerr = np.array([rsumCVal[1] for i in range(len(rsumCX))])

    print('rsum-Ch value %s = '%(ri), rsumCVal)

    if ri == 'Re':
        cfLbl = 'r-summ. const. fit (H)'
    else:
        cfLbl = 'none'
    axRS[iri][1].plot(rsumCX, rsumCY, color='rebeccapurple',ls='-',linewidth=1.0)
    axRS[iri][1].fill_between( rsumCX , rsumCY-rsumCYerr, rsumCY+rsumCYerr, facecolor='rebeccapurple', alpha=0.25,label=cfLbl)  
    #-----------------------------
    
    
#-------------------------------------------------------------------------------


for iri,ri in enumerate(ReIm):
    axRS[iri][0].set_ylabel(r'%s[$R_{\rm sum}(p_z = %s;z_3 = %s)$]'%(ri,mVecRS[2],dL),fontsize=18)
    axRS[iri][1].set_ylabel(r'%s[$R_{\rm r-sum}(p_z = %s;z_3 = %s)$]'%(ri,mVecRS[2],dL),fontsize=18)

    axRS[iri][0].set_xlim(2,16)
    axRS[iri][0].xaxis.set_ticks(np.arange(2, 17, 2))
    axRS[iri][1].set_xlim(2,14)
    axRS[iri][1].xaxis.set_ticks(np.arange(2, 15, 2))
    
    
axRS[1][0].set_xlabel(r'$t_s/a$',fontsize=18)
axRS[1][1].set_xlabel(r'$t_s/a$',fontsize=18)

axRS[0][0].legend(loc='best', ncol=1, numpoints=1, fontsize=15,
           labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
           markerscale=0.8)

axRS[0][1].legend(loc='best', ncol=2, numpoints=1, fontsize=15,
           labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
           markerscale=0.8)


figRS.savefig('summed_ratio_fits_%s_z%d.pdf'%(mTagRS,dL),bbox_inches='tight')



## Perform 2-state fits on the Ratio

These fits are according to the fitting function: $\frac{A_{00}+A_{11}e^{-\Delta Et_s}+A_{01}\left(e^{-\Delta Et_{ins}} + e^{-\Delta E(t_s-t_{ins})}\right)}{c_0+c_1 e^{-\Delta Et_s}}$,
 where $A_{00}$, $A_{01}$, $A_{11}$, $c_0$, $c_1$ and $\Delta E$ are fit parameters. The matrix element is obtained as $M = \frac{A_{00}}{c_0}$.

In [ ]:

#fitParams2st = ['A00','A01','A11','c0','c1','E0','E1']
fitParam2st = ['A00','A01','A11','c0','c1','dE']
Nfprm = len(fitParam2st)

twoSt_fR = [4,6,8,10]

M2st_bins = {}
M2st_ave  = {}
fit2st_bins = {}
fit2st_ave = {}
for itsT, tsT in enumerate(twoSt_fR):
    tsT_s = 'ts%d'%(tsT)

    fit2st_chiAve = {'Re': {}, 'Im': {}}

    fit2st_bins[tsT_s] = {}
    fit2st_ave[tsT_s]  = {}
    for fp in fitParam2st:
        fit2st_bins[tsT_s][fp] = {'Re': {}, 'Im': {}}
        fit2st_ave[tsT_s][fp]  = {'Re': {}, 'Im': {}}

    M2st_bins[tsT_s] = {'Re': {}, 'Im': {}}
    M2st_ave[tsT_s]  = {'Re': {}, 'Im': {}}



for opT in SrcSnkOpTypes:
    for phT in phaseTypes:
        ophTpl = (phT,opT)
        momTags = momTags_Dict_ave[ophTpl]
 
        for itsT, tsT in enumerate(twoSt_fR):
            tsT_s = 'ts%d'%(tsT)

            for fp in fitParam2st:
                fit2st_bins[tsT_s][fp] = {'Re': {}, 'Im': {}}
                fit2st_ave[tsT_s][fp]  = {'Re': {}, 'Im': {}}
            fit2st_chiAve[tsT_s] = {'Re': {}, 'Im': {}}

            M2st_bins[tsT_s] = {'Re': {}, 'Im': {}}
            M2st_ave[tsT_s]  = {'Re': {}, 'Im': {}}

            # Determine the number of fit points
            Tsink_2st =  TsinkList[(phT,opT)][itsT:]
            print(Tsink_2st)

            Nfpts = 0
            for tsnk in Tsink_2st:
                Nfpts += (tsnk-1) # (Exclude source-point)
            print('Nfpts = ',Nfpts)

            for mTag in momTags:#['pz.+2']:#momTags:
                mVec = momVector_Dict[mTag]
                zMom = mVec[2]
                mFTag = momFileTag(mVec)
                pmTpl = phMomTpl(phT,zMom)

                # Make sure number of bins is the same for all tsink
    #            Nbins = 50
                dTpl = (phT,opT,mTag,Tsink_2st[0])
                Nbins = Nbins_3pt_Dict[dTpl]
                for tsnk in Tsink_2st[1:]:
                    dTpl = (phT,opT,mTag,tsnk)
                    if  Nbins_3pt_Dict[dTpl] != Nbins:
                        raise ValueError('Inconsistency in Nbins for tsnk = ',tsnk)          

                srcOpList = srcOpList3pt_Dict[opT][pmTpl] # These are the same in two- and
                snkOpList = srcOpList3pt_Dict[opT][pmTpl] # three-point functions

                for isrc, src in enumerate(srcOpList):
                    for isnk, snk in enumerate(snkOpList):                        
                        # Only upolarized PDF for now
                        irowSrc = 1
                        irowSnk = 1
                        gMat = 'gt'

                        for z in dispLen_ave:#[3]:#dispLen_ave:

                            for ri in ReIm:
                                ckeyF = (opT,phT,mTag,isrc,isnk,gMat,z,irowSrc,irowSnk) # 3pt-tuple
                                if z*mVec[2]==0 and ri=='Im':
                                    print('Skipping %s:'%(ri), ckeyF)
                                    continue
                                else:

                                    M2st_bins[tsT_s][ri][ckeyF] = np.zeros(Nbins)

                                    for fp in fitParam2st:
                                        fit2st_bins[tsT_s][fp][ri][ckeyF] = np.zeros(Nbins)
                                    fit2st_chiBins = np.zeros(Nbins)                            
                                    fit2st_chiBinsAll = np.zeros(Nbins)

                                    mC = 0
                                    for nb in range(Nbins):
                                        XData  = np.zeros(Nfpts)
                                        fitVal = np.zeros(Nfpts)
                                        fitErr = np.zeros(Nfpts)
                                        TsVar  = np.zeros(Nfpts)

                                        fIdx = 0
                                        for itsnk, tsnk in enumerate(Tsink_2st):
                                            NtIns = tsnk-1

                                            adTplM = (phT,opT,mTag,tsnk)
                                            if analyzeDataSetM[adTplM]:
                                                ckey3 = (opT,phT,mTag,tsnk,isrc,isnk,gMat,z,irowSrc,irowSnk) # 3pt-tuple

                                                for tins in range(NtIns):
                                                    # Set fit values and error
                                                    XData[fIdx] = tins+1
                                                    fitVal[fIdx] = RA_bins[ri][ckey3][nb,tins+1]
                                                    fitErr[fIdx] = RA_ave[ri][ckey3][1][tins+1]

                                                    TsVar[fIdx] = tsnk # Tsink variable

                                                    fIdx += 1
                                                # for-tins
                                        # for-tsnk

                                        if mC==0:
                                            guess_2st = np.array([1.0 for i in range(Nfprm)])
                                        else:
                                            guess_2st = np.zeros(Nfprm)
                                            for ifp,fp in enumerate(fitParam2st):
                                                guess_2st[ifp] = fit2st_bins[tsT_s][fp][ri][ckeyF][mC-1]

                                        try:
                                            XDataTpl = (XData,TsVar)
                                            fParams_2st, covMat_2st = scipyOpt.curve_fit(twoStateFit,XDataTpl,fitVal,sigma=fitErr,p0=guess_2st)

                                            for ifp,fp in enumerate(fitParam2st):
                                                fit2st_bins[tsT_s][fp][ri][ckeyF][mC] = fParams_2st[ifp]

                                            fit2st_chiBinsAll[mC] = chiSq_twoState(XDataTpl,fitVal,fitErr,
                                                                                   fit2st_bins[tsT_s]['A00'][ri][ckeyF][mC],fit2st_bins[tsT_s]['A01'][ri][ckeyF][mC],fit2st_bins[tsT_s]['A11'][ri][ckeyF][mC],
                                                                                   fit2st_bins[tsT_s]['c0'][ri][ckeyF][mC] ,fit2st_bins[tsT_s]['c1'][ri][ckeyF][mC] ,fit2st_bins[tsT_s]['dE'][ri][ckeyF][mC])
                                            fit2st_chiBinsAll[mC] /= (Nfpts-Nfprm-1)

                                            # The matrix element
                                            Melem = 0.5*(fit2st_bins[tsT_s]['A00'][ri][ckeyF][mC] / 
                                                         fit2st_bins[tsT_s]['c0'][ri][ckeyF][mC] )

                                            if not np.isnan(Melem):
                                                M2st_bins[tsT_s][ri][ckeyF][mC] = Melem
                                                fit2st_chiBins[mC] = fit2st_chiBinsAll[mC]
                                                mC += 1
                                        except:
                                            pass
                                    # for-nbins

                                    for fp in fitParam2st:
                                         fit2st_bins[tsT_s][fp][ri][ckeyF] =  fit2st_bins[tsT_s][fp][ri][ckeyF][:mC]

                                    M2st_bins[tsT_s][ri][ckeyF] = M2st_bins[tsT_s][ri][ckeyF][:mC]
                                    fit2st_chiBins = fit2st_chiBins[:mC]

#                                     print(ckeyF)
#                                     print('M2st = ', M2st_bins[tsT_s][ri][ckeyF])
#                                     print('chi2 = ', fit2st_chiBins)

                                    M2st_ave[tsT_s][ri][ckeyF] = jackknife_ave(M2st_bins[tsT_s][ri][ckeyF],mC,tL=1)
                                    fit2st_chiAve[tsT_s][ri][ckeyF] = jackknife_ave(fit2st_chiBins,mC,tL=1)

                                    print('%s - mom = %s , z = %d: mC = %d: '%(ri,mTag,z,mC),M2st_ave[tsT_s][ri][ckeyF])#,fit2st_chiAve[tsT_s][ri][ckeyF])
                                # if pz*z == 0 and Imaginary
                            # Re-im

                print('Done: Momentum %s, ts_low = %d'%(mTag,tsT))
            # for-momentum
        # for two-state ts-ranges
        
print('\nDone\n')                                    


These fits are according to the fitting function: $\frac{M+A_{11}e^{-\Delta Et_s}+A_{01}\left(e^{-\Delta Et_{ins}} + e^{-\Delta E(t_s-t_{ins})}\right)}{1+c_1 e^{-\Delta Et_s}}$,
where $M$, $A_{01}$, $A_{11}$, $c_1$ and $\Delta E$ are fit parameters. In this case, $M$ is the desired matrix element.

In [ ]:

#fitParams2st = ['A00','A01','A11','c0','c1','E0','E1']
fitParam2stOpt = ['M','A01','A11','c1','dE']
Nfprm = len(fitParam2stOpt)

twoSt_fR = [4,6,8,10]

revTs = True

M2stOpt_bins = {}
M2stOpt_ave  = {}
fit2stOpt_bins = {}
fit2stOpt_ave = {}
for tsT in twoSt_fR:
    itsT = None
    tsT_s = 'ts%d'%(tsT)

    fit2stOpt_chiAve = {'Re': {}, 'Im': {}}

    fit2stOpt_bins[tsT_s] = {}
    fit2stOpt_ave[tsT_s]  = {}
    for fp in fitParam2stOpt:
        fit2stOpt_bins[tsT_s][fp] = {'Re': {}, 'Im': {}}
        fit2stOpt_ave[tsT_s][fp]  = {'Re': {}, 'Im': {}}

    M2stOpt_bins[tsT_s] = {'Re': {}, 'Im': {}}
    M2stOpt_ave[tsT_s]  = {'Re': {}, 'Im': {}}



for opT in SrcSnkOpTypes:
    for phT in phaseTypes:
        ophTpl = (phT,opT)
        momTags = momTags_Dict_ave[ophTpl]

        if revTs:
            twoSt_Iter = twoSt_fR[::-1]
        else:
            twoSt_Iter = twoSt_fR
        
        for tsT in twoSt_Iter:
            tsT_s = 'ts%d'%(tsT)

            for fp in fitParam2stOpt:
                fit2stOpt_bins[tsT_s][fp] = {'Re': {}, 'Im': {}}
                fit2stOpt_ave[tsT_s][fp]  = {'Re': {}, 'Im': {}}
            fit2stOpt_chiAve[tsT_s] = {'Re': {}, 'Im': {}}

            M2stOpt_bins[tsT_s] = {'Re': {}, 'Im': {}}
            M2stOpt_ave[tsT_s]  = {'Re': {}, 'Im': {}}
  
            for mTag in momTags:#['pz.+2']:#momTags:
                mSkip10 = (mTag == 'pz.+3' or mTag == 'pz.+2')
                if mSkip10 and tsT == 10:
                    print('Skipping tsLow = %d for %s'%(tsT,mTag))
                    continue
                
                mVec = momVector_Dict[mTag]
                zMom = mVec[2]
                mFTag = momFileTag(mVec)
                pmTpl = phMomTpl(phT,zMom)
                
                # Determine the number of fit points
                tsLow = TsinkList[(phT,opT)].index(tsT)
                if mTag == 'pz.+3':
                    Tsink_2st = TsinkList[(phT,opT)][tsLow:-1]
                else:
                    Tsink_2st = TsinkList[(phT,opT)][tsLow:]
                print(Tsink_2st)

                Nfpts = 0
                for tsnk in Tsink_2st:
                    Nfpts += (tsnk-1) # (Exclude source-point)
                print('Nfpts = ',Nfpts)

                
                # Make sure number of bins is the same for all tsink
                dTpl = (phT,opT,mTag,Tsink_2st[0])
                Nbins = Nbins_3pt_Dict[dTpl]
                for tsnk in Tsink_2st[1:]:
                    dTpl = (phT,opT,mTag,tsnk)
                    if  Nbins_3pt_Dict[dTpl] != Nbins:
                        raise ValueError('Inconsistency in Nbins for tsnk = ',tsnk)          

                srcOpList = srcOpList3pt_Dict[opT][pmTpl] # These are the same in two- and
                snkOpList = srcOpList3pt_Dict[opT][pmTpl] # three-point functions

                for isrc, src in enumerate(srcOpList):
                    for isnk, snk in enumerate(snkOpList):                        
                        # Only upolarized PDF for now
                        irowSrc = 1
                        irowSnk = 1
                        gMat = 'gt'

                        for z in dispLen_ave:#[3]:#dispLen_ave:

                            for ri in ReIm:
                                ckeyF = (opT,phT,mTag,isrc,isnk,gMat,z,irowSrc,irowSnk) # 3pt-tuple
                                if z*mVec[2]==0 and ri=='Im':
#                                    print('Skipping %s:'%(ri), ckeyF)
                                    continue
                                else:

                                    M2stOpt_bins[tsT_s][ri][ckeyF] = np.zeros(Nbins)

                                    for fp in fitParam2stOpt:
                                        fit2stOpt_bins[tsT_s][fp][ri][ckeyF] = np.zeros(Nbins)
                                    fit2st_chiBins = np.zeros(Nbins)                            
                                    fit2st_chiBinsAll = np.zeros(Nbins)

                                    mC = 0
                                    for nb in range(Nbins):
                                        XData  = np.zeros(Nfpts)
                                        fitVal = np.zeros(Nfpts)
                                        fitErr = np.zeros(Nfpts)
                                        TsVar  = np.zeros(Nfpts)

                                        fIdx = 0
                                        for itsnk, tsnk in enumerate(Tsink_2st):
                                            NtIns = tsnk-1

                                            adTplM = (phT,opT,mTag,tsnk)
                                            if analyzeDataSetM[adTplM]:
                                                ckey3 = (opT,phT,mTag,tsnk,isrc,isnk,gMat,z,irowSrc,irowSnk) # 3pt-tuple

                                                for tins in range(NtIns):
                                                    # Set fit values and error
                                                    XData[fIdx] = tins+1
                                                    fitVal[fIdx] = RA_bins[ri][ckey3][nb,tins+1]
                                                    fitErr[fIdx] = RA_ave[ri][ckey3][1][tins+1]

                                                    TsVar[fIdx] = tsnk # Tsink variable

                                                    fIdx += 1
                                                # for-tins
                                        # for-tsnk

                                        
                                        # Construct 2-state fit guesses
                                        guess_2st = np.array([1.0 for i in range(Nfprm)])
                                        
                                        if(mTag != 'pz.+2' and mTag != 'pz.+3'):
                                            tsT0 = twoSt_Iter[0]
                                            if revTs:
                                                chkCond = ( tsT != tsT0 and not (tsT==8 and mSkip10) )
                                            else:
                                                chkCond = ( tsT != tsT0 )
                                            if chkCond: # Use the previous ts_low value at same Pz, z3
                                                tsT_sG = 'ts%d'%(twoSt_Iter[twoSt_Iter.index(tsT)-1])
                                                ckeyG = (opT,phT,mTag,isrc,isnk,gMat,z,irowSrc,irowSnk)
                                                for ifp,fp in enumerate(fitParam2stOpt):
                                                    testG = fit2stOpt_ave[tsT_sG][fp][ri][ckeyG][0]
                                                    if testG != None:
                                                        guess_2st[ifp] = testG
                                            else:
                                                if z != 0:
                                                    zG  = z-1
                                                    zG0 = 0
                                                    if mTag != 'pz.0':
                                                        # Use (with priority):
                                                        # (a) Pz=0, same z3
                                                        # (b) Previous Pz, same z3
                                                        # (c) Same Pz, previous z3
                                                        mTagGp = MomTag(mVec[2]-1)
                                                        mTagG0 = MomTag(0)
                                                        ckeyGps = (opT,phT,mTagGp,isrc,isnk,gMat,z ,irowSrc,irowSnk)
                                                        ckeyG0s = (opT,phT,mTagG0,isrc,isnk,gMat,z ,irowSrc,irowSnk)
                                                        ckeyGsp = (opT,phT,mTagG ,isrc,isnk,gMat,zG,irowSrc,irowSnk)
                                                        for ifp,fp in enumerate(fitParam2stOpt):
                                                            try:
                                                                testGps = fit2stOpt_ave[tsT_s][fp][ri][ckeyGps][0]
                                                                testG0s = fit2stOpt_ave[tsT_s][fp][ri][ckeyG0s][0]
                                                                testGsp = fit2stOpt_ave[tsT_s][fp][ri][ckeyGsp][0]
                                                            except:
                                                                testGps = 1.0#fit2stOpt_ave[tsT_s][fp]['Re'][ckeyG][0]
                                                                testG0s = 1.0#fit2stOpt_ave[tsT_s][fp]['Re'][ckeyG0][0]
                                                                testGsp = 1.0#fit2stOpt_ave[tsT_s][fp]['Re'][ckeyG00][0]

                                                            if testG0s != None:
                                                                guess_2st[ifp] = testG0s
                                                            else:
                                                                if testGps != None:
                                                                    guess_2st[ifp] = testGps
                                                                else:
                                                                    if testGsp != None:
                                                                        guess_2st[ifp] = testGsp
                                                    else: # Use either previous z3, or z3=0, same Pz(=0)
                                                        ckeyG   = (opT,phT,mTag,isrc,isnk,gMat,zG ,irowSrc,irowSnk)
                                                        ckeyG0  = (opT,phT,mTag,isrc,isnk,gMat,zG0,irowSrc,irowSnk)
                                                        for ifp,fp in enumerate(fitParam2stOpt):
                                                            testG   = fit2stOpt_ave[tsT_s][fp][ri][ckeyG][0]
                                                            testG0  = fit2stOpt_ave[tsT_s][fp][ri][ckeyG0][0]
                                                            if testG != None:
                                                                guess_2st[ifp] = testG
                                                            else:
                                                                if testG0 != None:
                                                                    guess_2st[ifp] = testG0
                                                # if z != 0
                                                else:
                                                    if mTag != 'pz.0': # Use previous Pz, or Pz=0 same z3(=0)
                                                        mTagG  = MomTag(mVec[2]-1)
                                                        mTagG0 = MomTag(0)
                                                        ckeyG  = (opT,phT,mTagG,isrc,isnk,gMat,z,irowSrc,irowSnk)
                                                        ckeyG0 = (opT,phT,mTagG0,isrc,isnk,gMat,z,irowSrc,irowSnk)
                                                        for ifp,fp in enumerate(fitParam2stOpt):
                                                            testG  = fit2stOpt_ave[tsT_s][fp][ri][ckeyG][0]
                                                            testG0 = fit2stOpt_ave[tsT_s][fp][ri][ckeyG0][0]
                                                            if testG != None:
                                                                guess_2st[ifp] = testG
                                                            else:
                                                                if testG0 != None:
                                                                    guess_2st[ifp] = testG0
                                                    else: # Use default guess
                                                        for ifp,fp in enumerate(fitParam2stOpt):
                                                            guess_2st[ifp] = 1.0
                                            # if tsT != 0 
                                        # if pz.+2 or pz.+3   
                                        
                                        # Set the matrix element to the result of a good fit
                                        guess_2st[0] = Lsum_bins['M_l4'][ri][ckeyF][nb]
                                        #------------------
                                                                            
                                                
                                        try:
                                            XDataTpl = (XData,TsVar)
                                            fParams_2st, covMat_2st = scipyOpt.curve_fit(twoStateFitOpt,XDataTpl,fitVal,sigma=fitErr,p0=guess_2st)

                                            for ifp,fp in enumerate(fitParam2stOpt):
                                                fit2stOpt_bins[tsT_s][fp][ri][ckeyF][mC] = fParams_2st[ifp]

                                            fit2st_chiBinsAll[mC] = chiSq_twoStateOpt(XDataTpl,fitVal,fitErr,
                                                                                      fit2stOpt_bins[tsT_s]['M'][ri][ckeyF][mC] ,fit2stOpt_bins[tsT_s]['A01'][ri][ckeyF][mC],fit2stOpt_bins[tsT_s]['A11'][ri][ckeyF][mC],
                                                                                      fit2stOpt_bins[tsT_s]['c1'][ri][ckeyF][mC],fit2stOpt_bins[tsT_s]['dE'][ri][ckeyF][mC])
                                            fit2st_chiBinsAll[mC] /= (Nfpts-Nfprm-1)

                                            # The matrix element
                                            Melem = fit2stOpt_bins[tsT_s]['M'][ri][ckeyF][mC]

                                            if not np.isnan(Melem):
                                                M2stOpt_bins[tsT_s][ri][ckeyF][mC] = Melem
                                                fit2st_chiBins[mC] = fit2st_chiBinsAll[mC]
                                                mC += 1
                                        except:
                                            pass
                                    # for-nbins

                                    for fp in fitParam2stOpt:
                                        fit2stOpt_bins[tsT_s][fp][ri][ckeyF] =  fit2stOpt_bins[tsT_s][fp][ri][ckeyF][:mC]
                                        
                                    M2stOpt_bins[tsT_s][ri][ckeyF] = M2stOpt_bins[tsT_s][ri][ckeyF][:mC]
                                    fit2st_chiBins = fit2st_chiBins[:mC]

#                                     print(ckeyF)
#                                     print('M2st = ', M2stOpt_bins[tsT_s][ri][ckeyF])
#                                     print('chi2 = ', fit2st_chiBins)

                                    if mC == 0: # No successful fits
                                        M2stOpt_ave[tsT_s][ri][ckeyF] = (None,None)
                                        fit2stOpt_chiAve[tsT_s][ri][ckeyF] = (None,None)
                                        for fp in fitParam2stOpt:
                                            fit2stOpt_ave[tsT_s][fp][ri][ckeyF] = (None,None)
                                    elif mC == 1: # 1 successful fit
                                        M2stOpt_ave[tsT_s][ri][ckeyF] = (M2stOpt_bins[tsT_s][ri][ckeyF][0],0.0)
                                        fit2stOpt_chiAve[tsT_s][ri][ckeyF] = (fit2st_chiBins[0],0.0)
                                        for fp in fitParam2stOpt:
                                            fit2stOpt_ave[tsT_s][fp][ri][ckeyF] = (fit2stOpt_bins[tsT_s][fp][ri][ckeyF][0],0.0)
                                    else:
                                        M2stOpt_ave[tsT_s][ri][ckeyF] = jackknife_ave(M2stOpt_bins[tsT_s][ri][ckeyF],mC,tL=1)
                                        fit2stOpt_chiAve[tsT_s][ri][ckeyF] = jackknife_ave(fit2st_chiBins,mC,tL=1)
                                        for fp in fitParam2stOpt:
                                            fit2stOpt_ave[tsT_s][fp][ri][ckeyF] = jackknife_ave(fit2stOpt_bins[tsT_s][fp][ri][ckeyF],mC,tL=1)


                                    print('%s - mom = %s , z = %d: mC = %d: '%(ri,mTag,z,mC),M2stOpt_ave[tsT_s][ri][ckeyF], ckeyF)#,fit2stOpt_chiAve[tsT_s][ri][ckeyF])
                                # if pz*z == 0 and Imaginary
                            # Re-im

                print('Done: Momentum %s, ts_low = %d'%(mTag,tsT))
            # for-momentum
        # for two-state ts-ranges
        
print('\nDone\n')                                    



In [ ]:
ckey2st = ('single', 'unphased', 'pz.0', 0, 0, 'gt', 1, 1, 1)
ts_low = 'ts4'

# for fp in fitParam2st:#['A00','A11', 'c0','c1']:
# #     print('Re %s = '%(fp), fit2st_bins[fp]['Re'][ckey2st][-5:-1])
# #     print('Im %s = '%(fp), fit2st_bins[fp]['Im'][ckey2st][-5:-1])
#     print('Re %s = '%(fp), fit2st_bins[fp]['Re'][ckey2st][:5])
#     print('Im %s = '%(fp), fit2st_bins[fp]['Im'][ckey2st][:5])

#print(fit2stOpt_bins[ts_low]['M']['Re'][ckey2st])
#print(np.std(fit2stOpt_bins[ts_low]['M']['Re'][ckey2st])/np.sqrt(349))

#print(fit2st_bins[ts_low]['c0']['Im'][ckey2st])


# print('\n')
# print(fit2st_bins['A00']['Re'][ckey2st]/fit2st_bins['c0']['Re'][ckey2st])
# print(fit2st_bins['A11']['Re'][ckey2st]/fit2st_bins['c1']['Re'][ckey2st])

# print(fit2st_bins['A00']['Re'][ckey2st]/fit2st_bins['c0']['Re'][ckey2st]-fit2st_bins['A11']['Re'][ckey2st]/fit2st_bins['c1']['Re'][ckey2st])


# print('\n')
# print(fit2st_bins['A00']['Im'][ckey2st]/fit2st_bins['c0']['Im'][ckey2st])
# print(fit2st_bins['A11']['Im'][ckey2st]/fit2st_bins['c1']['Im'][ckey2st])

# print(fit2st_bins['A00']['Im'][ckey2st]/fit2st_bins['c0']['Im'][ckey2st]-fit2st_bins['A11']['Im'][ckey2st]/fit2st_bins['c1']['Im'][ckey2st])



## Multi-plots with all fits

#### Dictionary with 'eye' plateau ranges

In [ ]:
# Keep the same ranges for real, imaginary for now
eyePlatFit = {'Re': {4:0, 6:1, 8:1, 10:2, 12:3,14:4},
              'Im': {4:0, 6:1, 8:1, 10:2, 12:3,14:4}}

# twoSt_mrk = '>'
# twoSt_col = 'blueviolet'

# Lsum_mrk = 'H'
# Lsum_col = 'deeppink'

Lsum_mrk = '>'
Lsum_col = 'blueviolet'

twoSt_mrk = 'H'
twoSt_col = 'deeppink'


E1sum_mrk = '<'
E1sum_col = 'goldenrod'

E1rsum_mrk = 'v'
E1rsum_col = 'tomato'


yLimFct = {'pz.0' : {0:0.99, 1:1.01},
           'pz.+1': {0:0.99, 1:1.01},
           'pz.+2': {0:0.99, 1:1.01},
           'pz.+3': {0:0.90, 1:1.10}}


In [ ]:
ri = 'Re'

mTagR = 'pz.+1'
mVecR = momVector_Dict[mTagR]

twoSt_fR_Plot = twoSt_fR
if mTagR == 'pz.+2' or mTagR == 'pz.+3':
    try:
        twoSt_fR_Plot.remove(10)
    except:
        pass
    

if ri == 'Re':
    dispPlot = [0,1,2,3,4,5,6,7,8]
else:
    dispPlot = [1,2,3,4,5,6,7,8]
nFitT = 3
nDisp = len(dispPlot)
bAx = nDisp - 1
tAx = 0
lAx = 0

figFM, axFM = plt.subplots(nDisp, nFitT, sharex='col', sharey='row', figsize=[16,24])
figFM.subplots_adjust(hspace=0.00,wspace=0.03)#,left=0.8)

gs = gridspec.GridSpec(nDisp, nFitT, width_ratios=[1.5,0.8,1.1])


opTR = 'single'
phTR = 'unphased'
ophTpl = (phTR,opTR)
isrcR = 0
isnkR = 0
irowSrcR = 1
irowSnkR = 1
gMatR = 'gt'

tsinkPlat = 12
iFit = 3

plot_rSummE1 = True


for idL,dL in enumerate(dispPlot):
    for ifT in range(nFitT):
        axFM[idL][ifT] = plt.subplot(gs[ifT+nFitT*idL])

    
    # Plot the ratio for all tsinks (leftmost axis)
    axIdx = 0
    i = 0
    tsRng = {}
    
    raMaxVal = -1e-6
    raMinVal = 1e6
    for tsink in TsinkList[(phTR,opTR)]:

        tsRng[tsink] = np.arange(tsink) - tsink/2

        adTplM = (phTR,opTR,mTagR,tsink)
        if analyzeDataSetM[adTplM]:
            ckey = (opTR,phTR,mTagR,tsink,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)

            axFM[idL][axIdx].errorbar(tsRng[tsink][1:]+osDctF[i]/2,  RA_ave[ri][ckey][0][1:],
                         yerr = RA_ave[ri][ckey][1][1:],
                         color = cDct[i], marker = mDct[i], ls = 'none',
                         label=r'$t_s=%da$'%(tsink),
                         ms = 8.0, elinewidth=elw, capsize=ecs,fillstyle='full')
            i+=1        
            #--------
            
            tsnkMid = None
            for tins in range(1,tsink):
                if RA_ave[ri][ckey][0][tins] > raMaxVal:
                    raMaxVal = RA_ave[ri][ckey][0][tins]
                if RA_ave[ri][ckey][0][tins] < raMinVal:
                    raMinVal = RA_ave[ri][ckey][0][tins]
        #if-analyze
    # for tsink
            
    # limits
    axFM[idL][axIdx].xaxis.set_ticks(np.arange(-8, 9, 2))
    axFM[idL][axIdx].set_xlim(-8,8)
    
    # Set y-limits based on tsep=14 values and errors
    tsnkErr = 14
    ckeyErr = (opTR,phTR,mTagR,tsnkErr,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)
    errOff = RA_ave[ri][ckeyErr][1][tsnkErr//2]
    yLim0 = yLimFct[mTagR][0]*(raMinVal - errOff)
    yLim1 = yLimFct[mTagR][1]*(raMaxVal + errOff)
#    print(idL,raMinVal,raMaxVal,errOff,yLim0,yLim1)
    axFM[idL][axIdx].set_ylim(yLim0,yLim1)
    
    ylims = axFM[idL][0].get_ylim()    
    #----------------------------------------------------------------------------------------------
 
        
        
    # Plot plateau fits for all tsep (center axis)
    axIdx = 1
    for itsP, tsnkP in enumerate(TsinkList[ophTpl]):
        ckeyPlat = (opTR,phTR,mTagR,tsnkP,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)

        iFit = optPlatFit[ri][ckeyPlat]
        if iFit == None:
            iFit = eyePlatFit[ri][tsnkP]
    
        if tsnkP == 4:
            platVal = (RA_ave[ri][ckeyPlat][0][2],RA_ave[ri][ckeyPlat][1][2])
        else:
            platVal = Mplat_ave[ri][ckeyPlat][iFit]
        platLab = r'$t_s = %da$'%(tsnkP)

        pXrng = np.array([tsnkP])
        axFM[idL][axIdx].errorbar(pXrng, platVal[0], yerr = platVal[1],
                                  marker = mDct[itsP], color = cDct[itsP], ls='none',ms=7.0,
                                  elinewidth = elw, capsize=ecs, fillstyle='full')#,label=platLab)
        
    # limits
    axFM[idL][axIdx].set_xlim(2.5,15.5)
    axFM[idL][axIdx].set_ylim(ylims[0],ylims[1])    
    #----------------------------------------------------------------------------------------------

    
    
    # Plot summed ratio linear fits for all ts_low, 1-exp fits and two-state fits (rightmost axis)
    axIdx = 2
    
    # Summed ratio linear fits
    ckeySL = (opTR,phTR,mTagR,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)
    for isf,sf in enumerate(Lsum_fR):
        sfs = 'l%s'%(sf)

        sumLVal = Lsum_ave['M_%s'%(sfs)][ri][ckeySL]

        if isf == 0:
            sumLLab = 'Summ. linear'
        else:
            sumLLab = None        
        
        pXrng = np.array([sf])-0.15
        axFM[idL][axIdx].errorbar(pXrng, sumLVal[0], yerr = sumLVal[1],
                                  marker = Lsum_mrk, color = Lsum_col, ls='none',ms=7.0,
                                  elinewidth = elw, capsize=ecs, fillstyle='full',label = sumLLab)        
    #---------------------------------

    
    # Two-state fits
    ckey2st = (opTR,phTR,mTagR,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)
    if (dL*mVecR[2] != 0) or ri=='Re':
        for itsT, tsT in enumerate(twoSt_fR_Plot):
            tsT_s = 'ts%d'%(tsT)

            twoStVal = M2stOpt_ave[tsT_s][ri][ckey2st]        

            if itsT == 0:
                twoStLab = '2-state'
            else:
                twoStLab = None        

            if twoStVal[0] != None:

                pXrng = np.array([tsT])+0.15
                axFM[idL][axIdx].errorbar(pXrng, twoStVal[0], yerr = twoStVal[1],
                                          marker = twoSt_mrk, color = twoSt_col, ls='none',ms=7.0,
                                          elinewidth = elw, capsize=ecs, fillstyle='full',label = twoStLab)        
            else:
                print('Skipping 2-state fit for tS = %d'%(tsT), ckey2st)            
    else:
        print('Skipping 2-state fit for ', ckey2st)
    #---------------------------------

    
    # 1-exp fits for regular- and reduced-summed ratio
    ckeyE1 = (opTR,phTR,mTagR,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)
    tE1s = Lsum_fR[-1]+1
    axFM[idL][axIdx].axvline(tE1s,lw=0.7,color='k',ls='--')    

    # Regular-summed ratio 1-exp
    e1Idx = 1
    sumE1Val = E1sum_ave['M'][ri][ckeyE1]

    E1fLbl = 'Summ. 1-exp fit'

    pXrng = np.array([tE1s+e1Idx])
    axFM[idL][axIdx].errorbar(pXrng, sumE1Val[0], yerr = sumE1Val[1],
                              marker = E1sum_mrk, color = E1sum_col, ls='none',ms=7.0,
                              elinewidth = elw, capsize=ecs, fillstyle='full',label = E1fLbl)
    
    if sumE1Val[0] == None:
        print('Skipping summ. 1-exp fit for ', ckeyE1)
    else:
        if np.isnan(sumE1Val[0]):
            print('Skipping summ. 1-exp fit for ', ckeyE1)
    #---------------------------------

        
    # Reduced-summed ratio 1-exp
    if plot_rSummE1:
        e1Idx = 2
        rsumE1Val = E1rsum_ave['M'][ri][ckeyE1]

        rE1fLbl = 'r-Summ. 1-exp fit'

        pXrng = np.array([tE1s+e1Idx])
        axFM[idL][axIdx].errorbar(pXrng, rsumE1Val[0], yerr = rsumE1Val[1],
                                  marker = E1rsum_mrk, color = E1rsum_col, ls='none',ms=7.0,
                                  elinewidth = elw, capsize=ecs, fillstyle='full',label = rE1fLbl)
        
        if rsumE1Val[0] == None:
            print('Skipping r-summ. 1-exp fit for ', ckeyE1)
        else:
            if np.isnan(rsumE1Val[0]):
                print('Skipping r-summ. 1-exp fit for ', ckeyE1)
    #---------------------------------
                
    # limits
    tSR = Lsum_fR[0]-1
    axFM[idL][axIdx].set_xlim(tSR,tE1s+e1Idx+1)
    axFM[idL][axIdx].set_ylim(ylims[0],ylims[1])
    #----------------------------------------------------------------------------------------------

    
    # Plot plateau, summation and 2-state bands
    tsnkP = 12
    tsnkIdx = TsinkList[(phTR,opTR)].index(tsnkP)
    ckeyPlat = (opTR,phTR,mTagR,tsnkP,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)

    iFit = optPlatFit[ri][ckeyPlat]
    if iFit == None:
        iFit = eyePlatFit[ri][tsnkP]
    platVal = Mplat_ave[ri][ckeyPlat][iFit]

    tsnkLsum = 8
    sfs = 'l%d'%(tsnkLsum)
    ckeySL   = (opTR,phTR,mTagR,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)
    LsumVal = Lsum_ave['M_%s'%(sfs)][ri][ckeySL]

    for iAx in range(nFitT):
        xlims = axFM[idL][iAx].get_xlim()
        if iAx == 0:
            bOffsL = 0.4#(xlims[1] - xlims[0])*0.03
            bOffsR = 0
        elif iAx == nFitT-1:
            bOffsR = 0.3#(xlims[1] - xlims[0])*0.03
            bOffsL = 0
        else:
            bOffsL = 0
            bOffsR = 0
        
        ibF = 0
        bandX = np.arange(xlims[0]+bOffsL*ibF,xlims[1]-bOffsR*ibF+0.1,0.1)

        platY = np.array([platVal[0] for i in range(len(bandX))])
        platYerr = np.array([platVal[1] for i in range(len(bandX))])

        axFM[idL][iAx].plot(bandX, platY, ls='-', color=cDct[tsnkIdx],linewidth=1.0,alpha=0.5)
        axFM[idL][iAx].fill_between(bandX, platY-platYerr, platY+platYerr, facecolor=cDct[tsnkIdx], alpha=0.14)#,label=r'Plat. $t_s=%d$'%(tsinkPlat))  

        ibF += 1 
        bandX = np.arange(xlims[0]+bOffsL*ibF,xlims[1]-bOffsR*ibF+0.1,0.1)

        sumLY = np.array([LsumVal[0] for i in range(len(bandX))])
        sumLYerr = np.array([LsumVal[1] for i in range(len(bandX))])

        axFM[idL][iAx].plot(bandX, sumLY, ls='-', color=Lsum_col,linewidth=1.0,alpha=0.5)
        axFM[idL][iAx].fill_between(bandX, sumLY-sumLYerr, sumLY+sumLYerr, facecolor=Lsum_col, alpha=0.14)#,label=r'Plat. $t_s=%d$'%(tsinkPlat)) 
    #----------------------------------------------------------------------------------------------    
      
    
    
    # Remove ytick labels except for leftmost and bottom axes
    if idL != bAx:
        for iAx in range(nFitT):
            axFM[idL][iAx].xaxis.set_ticklabels([])


    axFM[idL][1].yaxis.set_ticklabels([])
    axFM[idL][2].yaxis.set_ticklabels([])

    
    # Create pz,z3 label/text, and plateau text
    cstLabY = ylims[1] - (ylims[1]-ylims[0])*0.15
    axFM[idL][0].text(3,cstLabY,r'$p_z = %d$ , $z_3 = %d$'%(mVecR[2],dL),fontsize=16)
    if idL == 0:
        axFM[idL][1].text(4,cstLabY,'Plateau fits',fontsize=16)

    

# X-labels
axFM[bAx][0].set_xlabel(r'$[t_{\rm ins} - t_s/2]/a$',fontsize=18)

axFM[bAx][1].xaxis.set_ticks(np.arange(4, 15, 2))
axFM[bAx][1].set_xlabel(r'$t_s/a$',fontsize=18)

axFM[bAx][2].xaxis.set_ticks(np.arange(4, 11, 2))
axFM[bAx][2].set_xlabel(r'$t_s^{\rm low}/a$',fontsize=18)





# Y-label
figFM.text(0.06,0.5, r'$\mathbf{%s}[R(p_z,z_3)]$' % (ri), ha="left", va="center", rotation=90,fontsize=18)
# axFM[2][0].set_ylabel(r'Re[$R(p_z=%d;z_3)$]'%(mVecR[2]),fontsize=18)
# axFM[2][1].set_ylabel(r'Im[$R(p_z=%d;z_3)$]'%(mVecR[2]),fontsize=18)

# Legend
axFM[tAx][0].legend(loc='lower left', ncol=3, numpoints=1, fontsize=14,
                    labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
                    markerscale=0.8)
axFM[tAx][2].legend(loc='best', ncol=2, numpoints=1, fontsize=14,
                    labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
                    markerscale=0.8)



figFM.savefig('matElem_Allfits_%s_%s.pdf'%(ri,mTagR),bbox_inches='tight')



## Multi-plots with ratio

In [ ]:
figRM, axRM = plt.subplots(5, 2, sharex='col', figsize=[15,18]) #, sharey='row'
figRM.subplots_adjust(hspace=0.00,wspace=0.2)#,right=0.8)


mTagR = 'pz.+2'
mVecR = momVector_Dict[mTagR]

opTR = 'single'
phTR = 'unphased'
ophTpl = (phTR,opTR)
isrcR = 0
isnkR = 0
irowSrcR = 1
irowSnkR = 1

gMatR = 'gt'

tsinkPlat = 12
iFit = 3

smRng = np.arange(-8,9)

for idL,dL in enumerate([0,2,4,6,8]):

    for iri,ri in enumerate(ReIm):
        
        # Plot regular summation-method band (slope)
        ckeyF = (opTR,phTR,mTagR,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)

        sumLVal = Lsum_ave['M_l8'][ri][ckeyF]

        sumLX = smRng
        sumLY = np.array([sumLVal[0] for i in range(len(sumLX))])
        sumLYerr = np.array([sumLVal[1] for i in range(len(sumLX))])

        if ri == 'Re':
            sLbl = 'summ. linear'
        else:
            sLbl = 'none'

        axRM[idL][iri].plot(sumLX, sumLY, color='darkgreen', ls ='-', linewidth=1.0)
        axRM[idL][iri].fill_between(sumLX , sumLY-sumLYerr, sumLY+sumLYerr, facecolor='darkgreen', alpha=0.18, label=sLbl)     
        #----------------------------------------------------------------------------------------------

        i = 0
        tsRng = {}
        for tsink in TsinkList[(phTR,opTR)]:

            tsRng[tsink] = np.arange(tsink) - tsink/2

            adTplM = (phTR,opTR,mTagR,tsink)
            if analyzeDataSetM[adTplM]:
                ckey = (opTR,phTR,mTagR,tsink,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)

                axRM[idL][iri].errorbar(tsRng[tsink][1:]+osDctF[i]/2,  RA_ave[ri][ckey][0][1:],
                                     yerr = RA_ave[ri][ckey][1][1:],
                                     color = cDct[i], marker = mDct[i], ls = 'none',
                                     label=r'$t_s=%d$'%(tsink),
                                     ms = 7.0, elinewidth=elw, capsize=ecs,fillstyle='full')
                i+=1
        # end-for tsink

        
        # Plot plateau-band
        ckeyFit = (opTR,phTR,mTagR,tsinkPlat,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)

        ti = fitInfo_RP[ophTpl][tsinkPlat]['nf=%d'%(iFit)]['tstart']
        tf = fitInfo_RP[ophTpl][tsinkPlat]['nf=%d'%(iFit)]['tstop']
        tsnkIdx = TsinkList[(phTR,opTR)].index(tsinkPlat)

        platVal = Mplat_ave[ri][ckeyFit][iFit]

        platX = (tsRng[tsinkPlat][ti:tf+1] + osDctF[tsnkIdx]/2)
        platY = np.array([platVal[0] for i in range(len(platX))])
        platYerr = np.array([platVal[1] for i in range(len(platX))])

        if ri == 'Re':
            pLbl = 'plat.'
        else:
            pLbl = 'none'        
            if(np.abs(platVal[0]-platVal[1])<0.008 ):
                axRM[idL][iri].axhline(0,ls='--',color='k',lw=1)
        
        axRM[idL][iri].plot(platX, platY, '%s-' % (cDct[tsnkIdx]),linewidth=2.0)
        axRM[idL][iri].fill_between( platX , platY-platYerr, platY+platYerr, facecolor=cDct[tsnkIdx], alpha=0.25, label=pLbl)        
        #----------------------------------------------------

        
        # Plot reduced-summed ratio band constant fit with all points
#         rsf = 'l'
#         dps = 'M'+rsf

#         ckeyF = (opTR,phTR,mTagR,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)

#         rsumCVal = Crsum_ave[dps][ri][ckeyF]

#         rsumCX = smRng
#         rsumCY = np.array([rsumCVal[0] for i in range(len(rsumCX))])
#         rsumCYerr = np.array([rsumCVal[1] for i in range(len(rsumCX))])

#         if ri == 'Re':
#             rsLbl = 'r-summ. const. (L)'
#         else:
#             rsLbl = 'none'
        
#         axRM[idL][iri].plot(rsumCX, rsumCY, 'k-',linewidth=1.0)
#         axRM[idL][iri].fill_between(rsumCX , rsumCY-rsumCYerr, rsumCY+rsumCYerr, facecolor='k', alpha=0.18, label=rsLbl)     
        #----------------------------------------------------------------------------------------------
                
        # Plot reduced-summed ratio constant fit ignoring lowest point
        rsf = 'h'
        dps = 'M_'+rsf
        ckeyF = (opTR,phTR,mTagR,isrcR,isnkR,gMatR,dL,irowSrcRS,irowSnkRS)

        rsumCVal = Crsum_ave[dps][ri][ckeyF]

        rsumCX = smRng
        rsumCY = np.array([rsumCVal[0] for i in range(len(rsumCX))])
        rsumCYerr = np.array([rsumCVal[1] for i in range(len(rsumCX))])

#        print('rsum-Ch value %s = '%(ri), rsumCVal)

        if ri == 'Re':
            cfLbl = 'r-summ. const. (H)'
        else:
            cfLbl = 'none'
        axRM[idL][iri].plot(rsumCX, rsumCY, color='rebeccapurple',ls='-',linewidth=1.0)
        axRM[idL][iri].fill_between( rsumCX , rsumCY-rsumCYerr, rsumCY+rsumCYerr, facecolor='rebeccapurple', alpha=0.18,label=cfLbl)  
        #-----------------------------

        #----------------------------------------------------------------------------------------------

        # X-limits
        axRM[idL][iri].set_xlim(-8,8)

        # Y-limits
        if ri == 'Re':
#             yLim0 = 0.9*(platVal[0]+platVal[1])
#             yLim1 = 1.1*(platVal[0]+platVal[1])
#             axRM[idL][0].set_ylim(yLim0,yLim1)

            # Text with z3 info
            zT_pos = (yLim1-yLim0)*0.12 + yLim0
            axRM[idL][iri].text(-7,zT_pos,r'$z_3 = %d$'%(dL),fontsize=16)            
#        else:
#             yLim0 = 0.5*(platVal[0]+platVal[1])
#             yLim1 = 1.5*(platVal[0]+platVal[1])
#             axRM[idL][1].set_ylim(yLim0,yLim1)


# X-labels
axRM[4][0].set_xlabel(r'$[(t_{\rm ins} -t_s)/2]/a$',fontsize=18)
axRM[4][1].set_xlabel(r'$[(t_{\rm ins} -t_s)/2]/a$',fontsize=18)


# Y-labels
axRM[2][0].set_ylabel(r'Re[$R(p_z=%d;z_3)$]'%(mVecR[2]),fontsize=18)
axRM[2][1].set_ylabel(r'Im[$R(p_z=%d;z_3)$]'%(mVecR[2]),fontsize=18)

# Legend
axRM[0][0].legend(loc='best', ncol=3, numpoints=1, fontsize=14,
           labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
           markerscale=0.8)


figRM.savefig('ratio_plateau_summation_p.%d.pdf'%(mVecR[2]),bbox_inches='tight')


### Plot rest-frame density $\mathcal{M}(p_z=0;z_3^2)$ using plateau fit values

In [ ]:

figRFD, axRFD = plt.subplots(figsize=[10,6.25])

opTRS = 'single'
phTRS = 'unphased'
isrcRS = 0
isnkRS = 0
irowSrcRS = 1
irowSnkRS = 1

gMatRS = 'gt'

mTagRS = 'pz.0'
nf=0

ri = 'Re'

xRng = np.array(dispLen_ave)

it = 0
for tsink in TsinkList[(phTRS,opTRS)]:
    adTplM = (phTRS,opTRS,mTagRS,tsink)
    if analyzeDataSetM[adTplM]:
    
        restFrD = (np.zeros(len(dispLen_ave)),np.zeros(len(dispLen_ave)))

        for idL,dL in enumerate(dispLen_ave):
            ckey3 = (opTRS,phTRS,mTagRS,tsink,isrcRS,isnkRS,gMatRS,dL,irowSrcRS,irowSnkRS)


            for i in range(2):
                restFrD[i][idL] = Mplat_ave[ri][ckey3][nf][i] / ZV[0]


        axRFD.errorbar(xRng+osDctF[it],  restFrD[0], yerr = restFrD[1],
                     color = cDct[it], marker = mDct[it], ls = 'none', label = r'$t_s = %d$'%(tsink),
                     ms = 8.0, elinewidth=elw, capsize=ecs,fillstyle='full')
        
#        print(restFrD[0])

        it+=1

axRFD.set_xlabel(r'$z_3/a$',fontsize=18)
axRFD.set_ylabel(r'%s[$\mathcal{M}_{\rm plat}(0,z_3^2)$]'%(ri),fontsize=18)

axRFD.xaxis.set_ticks(np.arange(-1, 9, 1))
#axRFD.set_ylim(0.5,1.2)
#axRFD.yaxis.set_ticks(np.arange(0.45, 0.86, 0.05))

axRFD.legend(loc='best', ncol=2, numpoints=1, fontsize=18,
          labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
          markerscale=0.8)


figRFD.savefig('rest_frame_density_plateau_%s.pdf'%(ri),bbox_inches='tight')



## Evaluate reduced ITD for $t_s = 10$ based on plateau fits

In [ ]:

tSink = 12
iFit = 3

tsnk = None

rITD_bins = {'Re':{}, 'Im':{}}
rITD_ave  = {'Re':{}, 'Im':{}}

for opT in SrcSnkOpTypes:
    for phT in phaseTypes:
        ophTpl = (phT,opT)
        momTags = momTags_Dict_ave[ophTpl]
                
        for mTag in momTags:
            mVec = momVector_Dict[mTag]
            zMom = mVec[2]
            mFTag = momFileTag(mVec)
            pmTpl = phMomTpl(phT,zMom)

            srcOpList = srcOpList3pt_Dict[opT][pmTpl] # These are the same in two- and
            snkOpList = srcOpList3pt_Dict[opT][pmTpl] # three-point functions

            dTpl = (phT,opT,mTag,tSink)
            Nbins = Nbins_3pt_Dict[dTpl]
            
            adTplM = (phT,opT,mTag,tSink)
            if analyzeDataSetM[adTplM]:
                isrc = 0
                isnk = 0

                # Only upolarized PDF for now
                irowSrc = 1
                irowSnk = 1
                gMat = 'gt'

                for z in dispLen_ave:
                    ckey = (opT,phT,mTag,tSink,isrc,isnk,gMat,z,irowSrc,irowSnk)

                    ckey_z0    = (opT,phT,mTag,tSink,isrc,isnk,gMat,0,irowSrc,irowSnk)
                    ckey_p0    = (opT,phT,'pz.0',tSink,isrc,isnk,gMat,z,irowSrc,irowSnk)
                    ckey_p0_z0 = (opT,phT,'pz.0',tSink,isrc,isnk,gMat,0,irowSrc,irowSnk)

                    rITD_bins['Re'][ckey] = ( (Mplat_bins['Re'][ckey][iFit]       / Mplat_bins['Re'][ckey_z0][iFit]) *
                                              (Mplat_bins['Re'][ckey_p0_z0][iFit] / Mplat_bins['Re'][ckey_p0][iFit]))

                    rITD_bins['Im'][ckey] = ( (Mplat_bins['Im'][ckey][iFit]       / Mplat_bins['Re'][ckey_z0][iFit]) *
                                              (Mplat_bins['Re'][ckey_p0_z0][iFit] / Mplat_bins['Re'][ckey_p0][iFit]))
                    
                    for ri in ReIm:
                        rITD_ave[ri][ckey] = jackknife_ave(rITD_bins[ri][ckey],Nbins,tL=1)

                    
                print('Done: Momentum %s'%(mTag))

print('\nDone\n')

In [ ]:

figITD, axITD     = plt.subplots(figsize=[10,6.25])
figITDnu, axITDnu = plt.subplots(figsize=[10,6.25])


opTRS = 'single'
phTRS = 'unphased'
isrcRS = 0
isnkRS = 0
irowSrcRS = 1
irowSnkRS = 1

ophTpl = (phTRS,opTRS)
momTags = momTags_Dict_ave[ophTpl]

gMatRS = 'gt'

tsink = 12

adTplM = (phTRS,opTRS,mTagRS,tsink)
if not analyzeDataSetM[adTplM]:
    raise ValueError('Will not analyze ', adTplM)

ri = 'Re'

xRng = np.array(dispLen_ave)
    
ip = 0
for mTag in momTags:
    mVec = momVector_Dict[mTag]
    
    rITD_plot = (np.zeros(len(dispLen_ave)),np.zeros(len(dispLen_ave)))
    for idL,dL in enumerate(dispLen_ave):
        ckey = (opTRS,phTRS,mTag,tsink,isrcRS,isnkRS,gMatRS,dL,irowSrcRS,irowSnkRS)
        
        nu = mVec[2]*dL*uM
        
        for i in range(2):
            rITD_plot[i][idL] = rITD_ave[ri][ckey][i]

        if idL==0:
            nLbl = r'$p_z = %d$'%(mVec[2])
        else:
            nLbl = ''
            
        axITDnu.errorbar(nu, rITD_ave[ri][ckey][0], yerr = rITD_ave[ri][ckey][1],
             color = cDct[ip], marker = mDct[ip], ls = 'none', label = nLbl,
             ms = 8.0, elinewidth=elw, capsize=ecs,fillstyle='full')
    #-------------------------------
    
    axITD.errorbar(xRng+osDctF[ip]/2,  rITD_plot[0], yerr = rITD_plot[1],
                 color = cDct[ip], marker = mDct[ip], ls = 'none', label = r'$p_z = %d$'%(mVec[2]),
                 ms = 8.0, elinewidth=elw, capsize=ecs,fillstyle='full')
    
    ip+=1

axITD.set_xlabel(r'$z_3/a$',fontsize=18)
axITD.set_ylabel(r'%s[$\mathcal{M}_{\rm plat}(p_z,z_3^2)$]'%(ri),fontsize=18)

axITD.xaxis.set_ticks(np.arange(-1, 9, 1))
axITDnu.xaxis.set_ticks(np.arange(-0.5, 6, 0.5))

axITDnu.set_xlabel(r'$\nu$',fontsize=18)
axITDnu.set_ylabel(r'%s[$\mathcal{M}_{\rm plat}(\nu,z_3^2)$]'%(ri),fontsize=18)

axITD.legend(loc='best', ncol=2, numpoints=1, fontsize=18,
          labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
          markerscale=0.8)

axITDnu.legend(loc='best', ncol=2, numpoints=1, fontsize=18,
          labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
          markerscale=0.8)


figITD.savefig('rITD_z3_plateau_%s.pdf'%(ri),bbox_inches='tight')
figITDnu.savefig('rITD_nu_plateau_%s.pdf'%(ri),bbox_inches='tight')



### Plot ratio $R(t_s,t_{\rm ins})$

In [ ]:
figR = [None, None]
axR  = [None, None]

figR[0], axR[0] = plt.subplots(figsize=[10,6.25])
figR[1], axR[1] = plt.subplots(figsize=[10,6.25])

Fct0 = {'Re': 0.95, 'Im': 0.85}
Fct1 = {'Re': 1.05, 'Im': 1.15}


opTR = 'single'
phTR = 'unphased'
ophTpl = (phTR,opTR)
isrcR = 0
isnkR = 0
irowSrcR = 1
irowSnkR = 1

gMatR = 'gt'

mTagR = 'pz.+3'

dL = 3

xRini = -8
xRfin = 9
xRoff = 0.3

xLo = 2+0.3*isf
xHi = 14-0.3*isf
xSt = (xHi-xLo)/10
sumLX = np.arange(xLo,14+0.01,xSt)

fitBandRng = np.arange(-8,10)

for iri,ri in enumerate(ReIm):
    i = 0
    tsRng = {}
    for tsink in TsinkList[(phTR,opTR)]:

        tsRng[tsink] = np.arange(tsink) - tsink/2

        adTplM = (phTR,opTR,mTagR,tsink)
        if analyzeDataSetM[adTplM]:
            ckey = (opTR,phTR,mTagR,tsink,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)

            axR[iri].errorbar(tsRng[tsink][1:]+osDctF[i]/2,  RA_ave[ri][ckey][0][1:],
                         yerr = RA_ave[ri][ckey][1][1:],
                         color = cDct[i], marker = mDct[i], ls = 'none',
                         label=r'$t_s=%d$'%(tsink),
                         ms = 8.0, elinewidth=elw, capsize=ecs,fillstyle='full')
            i+=1
    #-------------------------------------------------------------------------------
 

    # Plot plateau fit        
    tsinkPlat = 12
    iFit = 3

    ckeyFit = (opTR,phTR,mTagR,tsinkPlat,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)

    ti = fitInfo_RP[ophTpl][tsinkPlat]['nf=%d'%(iFit)]['tstart']
    tf = fitInfo_RP[ophTpl][tsinkPlat]['nf=%d'%(iFit)]['tstop']
    tsnkIdx = TsinkList[(phTR,opTR)].index(tsinkPlat)

    platVal = Mplat_ave[ri][ckeyFit][iFit]

    platX = (tsRng[tsinkPlat][ti:tf+1] + osDctF[tsnkIdx]/2)
    platY = np.array([platVal[0] for i in range(len(platX))])
    platYerr = np.array([platVal[1] for i in range(len(platX))])

    print('Plateau value %s = '%(ri), platVal)

    axR[iri].plot(platX, platY, ls='-', color = cDct[tsnkIdx],linewidth=2.0)
    axR[iri].fill_between(platX, platY-platYerr, platY+platYerr, facecolor=cDct[tsnkIdx], alpha=0.25,label=r'Plat. $t_s=%d$'%(tsinkPlat))  

    if ri == 'Im':
        if(np.abs(platVal[0]-platVal[1])<0.01 ):
            axR[iri].axhline(0,ls='--',color='k',lw=1)

    axR[iri].set_xlabel(r'$[t_{\rm ins} -t_s/2]/a$',fontsize=18)
    axR[iri].set_ylabel(r'%s[$R(p_z;z_3;t_{\rm ins},t_s)$]'%(ri),fontsize=18)
    #----------------------------------------------------------------------------------------------

    
    # Plot 1-exp summation method band
    ckeyF = (opTR,phTR,mTagR,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)

    E1sumVal = E1sum_ave['M'][ri][ckeyF]
    print('E1-summ. value %s = '%(ri), E1sumVal)

    iFc = 0
    xLo = xRini+xRoff*iFc
    xHi = xRfin-xRoff*iFc
    xSt = (xHi-xLo)/10
    
    E1sumX = np.arange(xLo,xRfin+0.01,xSt)
    E1sumY = np.array([E1sumVal[0] for i in range(len(E1sumX))])
    E1sumYerr = np.array([E1sumVal[1] for i in range(len(E1sumX))])

    E1sumLbl = 'Summ. 1-exp'

    axR[iri].plot(E1sumX, E1sumY,ls='-',color='slategray',linewidth=1.0)
    axR[iri].fill_between(E1sumX , E1sumY-E1sumYerr, E1sumY+E1sumYerr, facecolor='slategray', alpha=0.25, label=E1sumLbl)     
    #------------------------------------------------

    # Plot linear fit band
    fs = 8
    sfs = 'l%d'%(fs)
    ckeyF = (opTR,phTR,mTagR,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)

    LsumVal = Lsum_ave['M_%s'%(sfs)][ri][ckeyF]
    print('L-summ. value %s %s = '%(sfs, ri), LsumVal)

    iFc += 1
    xLo = xRini+xRoff*iFc
    xHi = xRfin-xRoff*iFc
    xSt = (xHi-xLo)/10

    LsumX = np.arange(xLo,xRfin+0.01,xSt)
    LsumY = np.array([LsumVal[0] for i in range(len(LsumX))])
    LsumYerr = np.array([LsumVal[1] for i in range(len(LsumX))])

    LsumLbl = r'Summ. linear $t_s^{\rm low} = %d$'%(fs)

    axR[iri].plot(LsumX, LsumY,ls='-',color='tomato',linewidth=1.0)
    axR[iri].fill_between(LsumX , LsumY-LsumYerr, LsumY+LsumYerr, facecolor='tomato', alpha=0.15, label=LsumLbl)     
    #------------------------------------------------

    
            
    # Plot 2-state band
    ckeyF = (opTR,phTR,mTagR,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)

    tsT = 8
    tsT_s = 'ts%d'%(tsT)
    
    if M2st_ave[tsT_s][ri][ckeyF][0] != None:
        twoStVal = M2st_ave[tsT_s][ri][ckeyF]
        print('2-state value %s = '%(ri), twoStVal)

        iFc += 1
        xLo = xRini+xRoff*iFc
        xHi = xRfin-xRoff*iFc
        xSt = (xHi-xLo)/10
        
        twoStX = np.arange(xLo,xRfin+0.01,xSt)
        twoStY = np.array([twoStVal[0] for i in range(len(twoStX))])
        twoStYerr = np.array([twoStVal[1] for i in range(len(twoStX))])

        twoStLbl = r'2-state, $t_s^{\rm low} = %d$'%(tsT)

        axR[iri].plot(twoStX, twoStY, ls='-', color='darkorchid',linewidth=1.0)
        axR[iri].fill_between(twoStX , twoStY-twoStYerr, twoStY+twoStYerr, facecolor='darkorchid', alpha=0.15, label=twoStLbl)
    else:
        print('Skipping 2-state fit for ', ckeyF)
    #------------------------------------------------

    
    axR[iri].legend(loc='best', ncol=3, numpoints=1, fontsize=18,
           labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
           markerscale=0.8)

    axR[iri].xaxis.set_ticks(np.arange(-8, 9, 2))
    axR[iri].set_xlim(-8,9)
    
#-------------------------------------------------------------------------------

    yLim0 = Fct0[ri]*(E1sumVal[0]-E1sumVal[1])
    yLim1 = Fct1[ri]*(E1sumVal[0]+E1sumVal[1])
    axR[iri].set_ylim(yLim0,yLim1)


